# Best Calendar Retry Attempt

In [2]:
%set_env SPARK_HOME=/opt/spark
import os
import findspark
os.environ['PYSPARK_SUBMIT_ARGS'] = """\
--master spark://10.62.42.243:7077 
--packages datastax:spark-cassandra-connector:2.3.1-s_2.11,anguenot:pyspark-cassandra:0.9.0
--driver-memory 25g
--executor-memory 15g
--conf spark.local.dir=/var/spark
--conf spark.network.timeout=420000
--conf spark.shuffle.registration.timeout=42000
--conf spark.driver.cores=4
--conf spark.executor.cores=8
--conf spark.driver.maxResultSize=38
pyspark-shell
"""

# --conf spark.cores.max=16

# --driver-memory 70g
# --executor-memory 25g
# --conf spark.executor.heartbeatInterval=42000
# --conf num-executors=4
# --conf spark.shuffle.service.enabled=true
# --conf spark.dynamicAllocation.executorIdleTimeout=42000
# --conf spark.dynamicAllocation.enabled=true
# --conf spark.dynamicAllocation.minExecutors=4

import pyspark
from pyspark import SparkContext, SparkConf
findspark.init()
sc = pyspark.SparkContext(appName="BestRetry")

env: SPARK_HOME=/opt/spark


In [3]:
sc._conf.set("spark.sql.execution.arrow.enabled", "true")
# sc._conf.set('spark.driver.maxResultSize', '32')
# sc._conf.set('spark.driver.cores', '4')
# sc._conf.set('spark.executor.cores', '7')
# # sc._conf.set('spark.driver.memory', '24g')
# # sc._conf.set('spark.executor.memory', '24g')
# sc._conf.set("spark.cores.max", 6)
sc._conf.set("spark.local.dir", "/var/spark")

# sc._conf.set("spark.network.timeout", "1000000s")
# sc._conf.set("spark.executor.heartbeatInterval", "1000000s")

# sc._conf.set("spark.cassandra.input.split.size_in_mb", 32)
# sc._conf.set("spark.cassandra.splitCount", 16)
# sc._conf.set("spark.cassandra.connection.host", "10.224.12.32")

# sc._conf.set("spark.cassandra.connection.host", "10.62.1.118")
# sc._conf.set("spark.cassandra.auth.username", "mlprw")
# sc._conf.set("spark.cassandra.auth.password", "q4RgwD$wK7*z")

sc._conf.set("spark.cassandra.connection.host", "10.47.65.95")
sc._conf.set("spark.cassandra.auth.username", "mlprw")
sc._conf.set("spark.cassandra.auth.password", "Df^9c2%znt7S")

sc._conf.getAll()

[('spark.driver.cores', '4'),
 ('spark.deploy.spreadOut', 'true'),
 ('spark.cassandra.auth.password', 'Df^9c2%znt7S'),
 ('spark.sql.execution.arrow.enabled', 'true'),
 ('spark.driver.maxResultSize', '38'),
 ('spark.local.dir', '/var/spark'),
 ('spark.files',
  'file:///root/.ivy2/jars/datastax_spark-cassandra-connector-2.3.1-s_2.11.jar,file:///root/.ivy2/jars/anguenot_pyspark-cassandra-0.9.0.jar,file:///root/.ivy2/jars/com.twitter_jsr166e-1.1.0.jar,file:///root/.ivy2/jars/org.joda_joda-convert-1.2.jar,file:///root/.ivy2/jars/io.netty_netty-all-4.0.33.Final.jar,file:///root/.ivy2/jars/commons-beanutils_commons-beanutils-1.9.3.jar,file:///root/.ivy2/jars/joda-time_joda-time-2.3.jar,file:///root/.ivy2/jars/org.scala-lang_scala-reflect-2.11.8.jar,file:///root/.ivy2/jars/commons-collections_commons-collections-3.2.2.jar,file:///root/.ivy2/jars/com.datastax.spark_spark-cassandra-connector_2.11-2.3.0.jar,file:///root/.ivy2/jars/net.razorvine_pyrolite-4.10.jar,file:///root/.ivy2/jars/net.razor

In [4]:
import pyspark
from pyspark.sql import SparkSession, SQLContext

spark = SparkSession.builder.\
    master("spark://10.62.42.243:7077").\
    appName("BestRetry").\
    config(conf=sc._conf).\
    getOrCreate()
# spark

sqlContext = SQLContext(sc)
sqlContext

In [8]:
import time

import pandas as pd


from pyspark.sql.types import *
from pyspark.sql.functions import col, count, rand, collect_list, explode, struct, count, lit, lower
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.functions import udf
from pyspark.sql.types import *
import pyspark_cassandra
        
import calendar, datetime
from datetime import timedelta

WORK_DIR = '/var/spark/ml_files/'


def convert_status(value):
    if value == 'Completed':
        return 1
    else:
        return 0
    
    
def cast_to_int(value, default=-1):
    try:
        return int(value)
    except (ValueError, TypeError):
        return default


#Convert from str to datetime
def to_date(datestr):
    struct = time.strptime(datestr, "%Y-%m-%d %H:%M:%S")
    date = datetime.date(struct.tm_year,struct.tm_mon,struct.tm_mday)
    return date

#Get difference between d2 and d1 in days.
def days_between(d1, d2):
    d1 = to_date(d1)
    d2 = to_date(d2)
    return abs((d2 - d1).days)

#Determines the week (number) of the month
def week_of_month(datestr):
    date = to_date(datestr)
    #Calendar object. 6 = Start on Sunday, 0 = Start on Monday
    cal_object = calendar.Calendar(6)
    month_calendar_dates = cal_object.itermonthdates(date.year,date.month)

    day_of_week = 1
    week_number = 1

    for day in month_calendar_dates:
        #add a week and reset day of week
        if day_of_week > 7:
            week_number += 1
            day_of_week = 1

        if date == day:
            break
        else:
            day_of_week += 1

    return str(week_number)

def to_weekday(datestr):
    struct = time.strptime(datestr, "%Y-%m-%d %H:%M:%S")
    date = datetime.date(struct.tm_year,struct.tm_mon,struct.tm_mday)
    return date.isoweekday()

def daterange(date1_str, date2_str):
    start_date = to_date(date1_str + ' 00:00:00')
    end_date = to_date(date2_str + ' 00:00:00')
    if start_date > end_date:
        raise ValueError('start date cannot be after the end date.')
    for n in range(int((end_date - start_date).days) + 1):
        yield str(start_date + timedelta(n))
            
def date_range_condition_values(start_date, end_date):
    dates = daterange(start_date, end_date)
    for idx, date in enumerate(dates):
            yield date

df_decline_type = pd.read_csv(WORK_DIR + 'Decline_Type.csv')

msg_group = { 'declined' : 'decline', 
             'do_not_honor' : 'do not honor', 
             'txn_refused' : 'refuse', 
             'attempt_lower_amount' : 'lower amount',
            'Insufficient Funds' : 'insufficient',
            'not_allowed' : 'not allowed',
            'correct_cc_retry' : 'correct card',
            'invalid_cc' : 'invalid card',
            'lost_stolen' : 'lost or stolen',
            'invalid_account' : 'invalid account',
            'do_not_try' : 'do not try',
            'expired_card' : 'expired',
            'pickup_card' : 'pick',
            'blocked_first_used' : 'blocked',
            'invalid_txn' : 'invalid trans',
            'restricted_card' : 'restricted',
            'not_permitted' : 'not permitted',
            'expired card' : 'expired card',
            'unable to determine format' : 'determine format',
            'system error' : 'error',
            'no reply' : 'no reply',
             'no charge model found' : 'no charge model found',
             'issuer unavailable' : 'issuer unavailable',
             'litle http response code' : 'litle http response code',
             'corrupt input data to server' : 'corrupt input data to server',
             'ioexception' : 'ioexception',
             'invalid merchant' : 'invalid merchant',
            }

def group_response_msg(msg):
    other = 'Base'
    if isinstance(msg, str) == False:
        return other
    
    msg_lower = msg.lower()
    for key, val in msg_group.items():
        if val in msg_lower:
            return key
        
    return other   



def merge_by_sub(group):
   
    #Assign default values to additonal columns
    group['failed_attempt_date'] = ''
    group['failed_day_of_month'] = 0
    group['failed_response_code'] = ''
    group['failed_response_message'] = ''
    group['failed_decline_type'] = ''
    group['days_between'] = 0
    group['failed_event_uuid'] = ''    
    
    last = pd.DataFrame() 
    df_success = group[group['success'] == 1]

    if not df_success.empty:
        df_last_attempt = df_success.iloc[0]
    else:
        sorted_group = group.sort_values(by=['transaction_date_in_string', 'cid'])
        df_last_fail = sorted_group.tail(1)
        if not df_last_fail.empty:
            df_last_attempt =  df_last_fail.iloc[0]
        else:
            group['event_uuid'] = '' 
            return group
    
    df_failed_txn = group[group['event_uuid'] != df_last_attempt['event_uuid']]
    if df_failed_txn.empty:
        group['event_uuid'] = '' 
        return group
    
    last = last.append( [df_last_attempt] * len(df_failed_txn),ignore_index=True ).set_index(df_failed_txn.index)
        
    failed_attempt_date = df_failed_txn['transaction_date_in_string']
    
    last['failed_attempt_date'] = failed_attempt_date
    last['failed_day_of_month'] = df_failed_txn['day_of_month']
    last['day_of_month'] = last['day_of_month'].astype(int)
    last['failed_response_code'] = df_failed_txn['response_code']
    
    failed_response_msg = df_failed_txn['response_message']
    last['failed_response_message'] = failed_response_msg  
    
    last['failed_decline_type'] = df_failed_txn['response_message'].apply(decline_type) 
    last['days_between'] = last.apply(days_between_ds, axis=1)    
    last['failed_event_uuid'] =   df_failed_txn['event_uuid']   
        
    return last


def decline_type(response_msg):
    '''Converts to decline_type based on the given response_msg'''
    dec_type = df_decline_type[df_decline_type['DECLINE_TEXT'] == response_msg]['DECLINE_TYPE']
    if dec_type.empty or dec_type.iloc[0] == 'Base' :
        return group_response_msg(response_msg)
    else:
        return dec_type.iloc[0]

#Convert from str to datetime
def to_date(datestr):
    struct = time.strptime(datestr, "%Y-%m-%d %H:%M:%S")
    date = datetime.date(struct.tm_year,struct.tm_mon,struct.tm_mday)
    return date

#Get difference between d2 and d1 in days.
def days_between(d1, d2):
    d1 = to_date(d1)
    d2 = to_date(d2)

    return abs((d2 - d1).days)

#Get difference between d2 and d1 in days.
def days_between_ds(df):
    d1 = to_date(df['failed_attempt_date'])
    d2 = to_date(df['transaction_date_in_string'])
    return abs((d2 - d1).days)

    
udfValueToStatus = udf(convert_status, IntegerType())  

createDDL = """CREATE OR REPLACE TEMPORARY VIEW cpg_transaction_history
     USING org.apache.spark.sql.cassandra
     OPTIONS (
     table "cpg_transaction_history",
     keyspace "dev_sarsks",
     cluster "MLP Cluster",
     pushdown "true")"""
spark.sql(createDDL)

# df = spark.sql("SELECT * FROM cpg_transaction_history WHERE received_date like '2018-12-%' ")
df = spark.sql("SELECT * FROM cpg_transaction_history WHERE received_date like '2018-01-%' or received_date like '2018-02-%' or received_date like '2018-03-%' or received_date like '2018-04-%' or received_date like '2018-05-%' or received_date like '2018-06-%' or received_date like '2018-07-%' or received_date like '2018-08-%' or received_date like '2018-09-%' or received_date like '2018-10-%' or received_date like '2018-11-%' or received_date like '2018-12-%' ")

sdf =  df\
    .filter("!(renew_att_num == '1' and new_status in ('Completed') and cid is null and status == 'New')")\
    .withColumn("success", udfValueToStatus("new_status"))\
    .withColumn("day_of_month", df["day_of_month"].cast(IntegerType()))\
    .withColumn("day_of_week", lower(col("day_of_week")))\
    .cache()

print(sdf.schema)
    
grouped_schema = sdf.schema.add("failed_attempt_date", StringType())\
                            .add("failed_day_of_month", IntegerType())\
                            .add("failed_response_code", StringType())\
                            .add("failed_response_message", StringType())\
                            .add("failed_decline_type", StringType())\
                            .add("days_between", IntegerType())\
                            .add("failed_event_uuid", StringType())
                            
                            
@pandas_udf(grouped_schema, PandasUDFType.GROUPED_MAP)
def merge_by_sub_spark(spark_df):
    pdf = pd.DataFrame(spark_df)
    result = pdf.groupby(['subscription_id', 'subsegment_id']).apply(merge_by_sub)
    return result                            
                            
time_start = time.time()

def _map_to_pandas(rdds):
    """ Needs to be here due to pickling issues """
    return [pd.DataFrame(list(rdds))]

def toPandas(df, n_partitions=None):
    """
    Returns the contents of `df` as a local `pandas.DataFrame` in a speedy fashion. The DataFrame is
    repartitioned if `n_partitions` is passed.
    :param df:              pyspark.sql.DataFrame
    :param n_partitions:    int or None
    :return:                pandas.DataFrame
    """
    if n_partitions is not None: df = df.repartition(n_partitions)
    df_pand = df.rdd.mapPartitions(_map_to_pandas).collect()
    df_pand = pd.concat(df_pand)
    df_pand.columns = df.columns
    return df_pand

sdf_filtered = sdf\
        .filter(sdf.renew_att_num.isNotNull())\
        .groupby("subscription_id", "subsegment_id")\
        .apply(merge_by_sub_spark)\
        .filter("event_uuid != ''")  #.toPandas()

# pdf_all = toPandas(sdf_filtered, 64)  
pdf_all = sdf_filtered.toPandas()
print("# total time:", time.time() - time_start)      # total time: 2364.714904308319
    
pdf_all.shape

StructType(List(StructField(received_date,DateType,true),StructField(site_id,StringType,true),StructField(subscription_id,StringType,true),StructField(transaction_id,StringType,true),StructField(transaction_date,TimestampType,true),StructField(added_expiry_years,StringType,true),StructField(bank_code,StringType,true),StructField(bank_name,StringType,true),StructField(billing_country,StringType,true),StructField(bin,StringType,true),StructField(card_brand,StringType,true),StructField(card_category,StringType,true),StructField(card_class,StringType,true),StructField(card_usage,StringType,true),StructField(cc_expiration_date,StringType,true),StructField(cid,StringType,true),StructField(date_increment,StringType,true),StructField(day_of_month,IntegerType,true),StructField(day_of_week,StringType,true),StructField(event_uuid,StringType,true),StructField(exp_setting_id,StringType,true),StructField(funding_source,StringType,true),StructField(issuer_country,StringType,true),StructField(merchant

KeyboardInterrupt: 

In [6]:
# pdf_all.to_csv(WORK_DIR + 'retry_success_12.csv')
# pdf_all.to_csv(WORK_DIR + 'retry_success_01_to_12.csv')

In [5]:
pdf_all['success'].value_counts(normalize=True)

0    0.845805
1    0.154195
Name: success, dtype: float64

In [12]:
import time

import pandas as pd


from pyspark.sql.types import *
from pyspark.sql.functions import col, count, rand, collect_list, explode, struct, count, lit, lower
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.functions import udf
from pyspark.sql.types import *
import pyspark_cassandra
        
import calendar, datetime
from datetime import timedelta

WORK_DIR = '/var/spark/ml_files/'
# start_date = '2018-04-01'
# end_date = '2018-06-30'

def convert_status(value):
    if value == 'Completed' or value == 'Reversed':
        return 1
    else:
        return 0
    
    
def cast_to_int(value, default=-1):
    try:
        return int(value)
    except (ValueError, TypeError):
        return default


#Convert from str to datetime
def to_date(datestr):
    struct = time.strptime(datestr, "%Y-%m-%d %H:%M:%S")
    date = datetime.date(struct.tm_year,struct.tm_mon,struct.tm_mday)
    return date

#Get difference between d2 and d1 in days.
def days_between(d1, d2):
    d1 = to_date(d1)
    d2 = to_date(d2)
    return abs((d2 - d1).days)

#Determines the week (number) of the month
def week_of_month(datestr):
    date = to_date(datestr)
    #Calendar object. 6 = Start on Sunday, 0 = Start on Monday
    cal_object = calendar.Calendar(6)
    month_calendar_dates = cal_object.itermonthdates(date.year,date.month)

    day_of_week = 1
    week_number = 1

    for day in month_calendar_dates:
        #add a week and reset day of week
        if day_of_week > 7:
            week_number += 1
            day_of_week = 1

        if date == day:
            break
        else:
            day_of_week += 1

    return str(week_number)

def to_weekday(datestr):
    struct = time.strptime(datestr, "%Y-%m-%d %H:%M:%S")
    date = datetime.date(struct.tm_year,struct.tm_mon,struct.tm_mday)
    return date.isoweekday()

def daterange(date1_str, date2_str):
    start_date = to_date(date1_str + ' 00:00:00')
    end_date = to_date(date2_str + ' 00:00:00')
    if start_date > end_date:
        raise ValueError('start date cannot be after the end date.')
    for n in range(int((end_date - start_date).days) + 1):
        yield str(start_date + timedelta(n))
            
def date_range_condition_values(start_date, end_date):
    dates = daterange(start_date, end_date)
    for idx, date in enumerate(dates):
        date_formatted = date.replace('-','')
        yield date_formatted

df_decline_type = pd.read_csv(WORK_DIR + 'Decline_Type.csv')

msg_group = { 'declined' : 'decline', 
             'do_not_honor' : 'do not honor', 
             'txn_refused' : 'refuse', 
             'attempt_lower_amount' : 'lower amount',
            'Insufficient Funds' : 'insufficient',
            'not_allowed' : 'not allowed',
            'correct_cc_retry' : 'correct card',
            'invalid_cc' : 'invalid card',
            'lost_stolen' : 'lost or stolen',
            'invalid_account' : 'invalid account',
            'do_not_try' : 'do not try',
            'expired_card' : 'expired',
            'pickup_card' : 'pick',
            'blocked_first_used' : 'blocked',
            'invalid_txn' : 'invalid trans',
            'restricted_card' : 'restricted',
            'not_permitted' : 'not permitted',
            'expired card' : 'expired card',
            'unable to determine format' : 'determine format',
            'system error' : 'error'
            }

def group_response_msg(msg):
    other = 'Base'
    if isinstance(msg, str) == False:
        return other
    
    msg_lower = msg.lower()
    for key, val in msg_group.items():
        if val in msg_lower:
            return key
        
    return other   



def merge_by_sub(group):

    
    #Assign default values to additonal columns
    group['failed_attempt_date'] = ''
    group['failed_day_of_month'] = 0
    group['failed_response_code'] = ''
    group['failed_response_message'] = ''
    group['failed_decline_type'] = ''
    group['days_between'] = 0
    group['failed_event_uuid'] = ''    
    
    last = pd.DataFrame() 
    df_success = group[group['success'] == 1]

    if not df_success.empty:
        df_last_attempt = df_success.iloc[0]
    else:
        sorted_group = group.sort_values(by=['transaction_date_in_string', 'cid'])
        df_last_fail = sorted_group.tail(1)
        if not df_last_fail.empty:
            df_last_attempt =  df_last_fail.iloc[0]
        else:
            group['event_uuid'] = '' 
            return group
    
    
    df_failed_txn = group[group['event_uuid'] != df_last_attempt['event_uuid']]
    if df_failed_txn.empty:
        group['event_uuid'] = '' 
        return group
    
    last = last.append( [df_last_attempt] * len(df_failed_txn),ignore_index=True ).set_index(df_failed_txn.index)
        
    failed_attempt_date = df_failed_txn['transaction_date_in_string']
    
    last['failed_attempt_date'] = failed_attempt_date
    last['failed_day_of_month'] = df_failed_txn['day_of_month']
    last['day_of_month'] = last['day_of_month'].astype(int)
    last['failed_response_code'] = df_failed_txn['response_code']
    
    failed_response_msg = df_failed_txn['response_message']
    last['failed_response_message'] = failed_response_msg  
    
    last['failed_decline_type'] = df_failed_txn['response_message'].apply(decline_type) 
    last['days_between'] = last.apply(days_between_ds, axis=1)    
    last['failed_event_uuid'] =   df_failed_txn['event_uuid']   
        
    return last

def decline_type(response_msg):
    '''Converts to decline_type based on the given response_msg'''
    dec_type = df_decline_type[df_decline_type['DECLINE_TEXT'] == response_msg]['DECLINE_TYPE']
    if dec_type.empty or dec_type.iloc[0] == 'Base' :
        return group_response_msg(response_msg)
    else:
        return dec_type.iloc[0]

#Convert from str to datetime
def to_date(datestr):
    struct = time.strptime(datestr, "%Y-%m-%d %H:%M:%S")
    date = datetime.date(struct.tm_year,struct.tm_mon,struct.tm_mday)
    return date

#Get difference between d2 and d1 in days.
def days_between(d1, d2):
    d1 = to_date(d1)
    d2 = to_date(d2)

    return abs((d2 - d1).days)

#Get difference between d2 and d1 in days.
def days_between_ds(df):
    d1 = to_date(df['failed_attempt_date'])
    d2 = to_date(df['transaction_date_in_string'])
    return abs((d2 - d1).days)

# @pandas_udf(sdf.schema, PandasUDFType.GROUPED_MAP)

        

# date_list = list(date_range_condition_values(start_date, end_date))


udfValueToStatus = udf(convert_status, IntegerType())  

createDDL = """CREATE OR REPLACE TEMPORARY VIEW cpg_transaction_history
     USING org.apache.spark.sql.cassandra
     OPTIONS (
     table "cpg_transaction_history",
     keyspace "dev_sarsks",
     cluster "MLP Cluster",
     pushdown "true")"""
spark.sql(createDDL)

df = spark.sql("SELECT * FROM cpg_transaction_history")

sdf =  df\
    .filter("!(renew_att_num == '1' and new_status in ('Completed','Reversed') and cid is null and status == 'New')")\
    .withColumn("success", udfValueToStatus("new_status"))\
    .withColumn("day_of_month", df["day_of_month"].cast(IntegerType()))\
    .withColumn("day_of_week", lower(col("day_of_week")))\
    .cache()

print(sdf.schema)
    
grouped_schema = sdf.schema.add("failed_attempt_date", StringType())\
                            .add("failed_day_of_month", IntegerType())\
                            .add("failed_response_code", StringType())\
                            .add("failed_response_message", StringType())\
                            .add("failed_decline_type", StringType())\
                            .add("days_between", IntegerType())\
                            .add("failed_event_uuid", StringType())
                    
                            
@pandas_udf(grouped_schema, PandasUDFType.GROUPED_MAP)
def merge_by_sub_spark(spark_df):
    pdf = pd.DataFrame(spark_df)
    result = pdf.groupby(['subscription_id', 'subsegment_id']).apply(merge_by_sub)
    return result                            
                            
time_start = time.time()

year=2018
months = ['04','05','06']
for m in months:
    yearmonth = str('%s-%s'%(year,m))
    condition = str("received_date like '%s%%'" % yearmonth)
    filename = str("retry_success_%s.csv" % yearmonth)
    sdf_by_month = sdf\
        .where(condition)\
        .filter(sdf.renew_att_num.isNotNull())\
        .groupby("subscription_id", "subsegment_id")\
        .apply(merge_by_sub_spark)\
        .filter("event_uuid != ''")
    
#     sdf_by_month.toPandas().to_csv(WORK_DIR + filename) 
    toPandas(sdf_by_month, 4).to_csv(WORK_DIR + filename) 
    
    
print("# total time:", time.time() - time_start)    
    
# pdf_all

StructType(List(StructField(received_date,DateType,true),StructField(site_id,StringType,true),StructField(subscription_id,StringType,true),StructField(transaction_id,StringType,true),StructField(transaction_date,TimestampType,true),StructField(added_expiry_years,StringType,true),StructField(bank_code,StringType,true),StructField(bank_name,StringType,true),StructField(billing_country,StringType,true),StructField(bin,StringType,true),StructField(card_brand,StringType,true),StructField(card_category,StringType,true),StructField(card_class,StringType,true),StructField(card_usage,StringType,true),StructField(cc_expiration_date,StringType,true),StructField(cid,StringType,true),StructField(date_increment,StringType,true),StructField(day_of_month,IntegerType,true),StructField(day_of_week,StringType,true),StructField(event_uuid,StringType,true),StructField(exp_setting_id,StringType,true),StructField(funding_source,StringType,true),StructField(issuer_country,StringType,true),StructField(merchant

ValueError: Length mismatch: Expected axis has 0 elements, new values have 53 elements

In [13]:
sc.stop()

In [9]:
# pdf_all['day_of_week'].value_counts(dropna=False)
# pdf_all.to_csv('/drhdev/workspace/ml/pdf_all_site_2018-01-01_2018-06-30.csv')
from IPython.display import display
pd.options.display.max_columns = None
display(pdf_all)

,received_date,site_id,subscription_id,event_uuid,added_expiry_years,bank_code,bank_name,billing_country,bin,card_brand,card_category,card_class,card_usage,cc_expiration_date,cid,date_increment,day_of_month,day_of_week,exp_setting_id,funding_source,issuer_country,merchant_descriptor,merchant_number,mid_entity_code,new_status,order_entity_code,payment_amount,payment_amount_usd,payment_currency,payment_method_id,payment_service_id,renew_att_num,request_amount,request_amount_usd,request_currency,requisition_id,response_code,response_message,retry_rule_id,status,subsegment_id,time_of_day_rule_id,transaction_date,transaction_date_in_string,transaction_hour,transaction_id,transaction_type,success,first_attempt_date,first_day_of_month,first_transaction_hour,first_response_code,first_response_message,first_response_group,first_decline_type,days_between
0,20180418,avast,10001280501,673671cc-cc57-400c-b872-8ff9efb09a6d,None,NON3DS,Standard Chartered Bank Tanzania Limited,TZ,478680,Visa,Visa Gold,Consumer,NON USA Consumer Debit - No Pin Access,0322,None,None,18,wednesday,None,Debit,TZ,DRI*AVAST Software,1411163460,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,79.99,79.99,USD,Visa,netgiro-bms,3,79.99,79.99,USD,46971082700,27050,Do not try again/use alternate payment card.[0...,None,New,13258980000,,2018-04-18 06:21:29,2018-04-18 01:21:29,01,6681738323,Authorize,0,2018-04-03 01:07:59,3,01,27050,Do not try again/use alternate payment card.[0...,do_not_try,Hard Fraud,15
1,20180418,avastbr,10001441601,11cc8ab0-fe20-4e80-9d8c-2a7a856e7f17,None,NON3DS,ITAU UNIBANCO S.A.,BR,539083,MasterCard,(Unembossed) Standard MasterCard Card,Consumer,True credit (No PIN/Signature capability),1223,None,None,18,wednesday,None,Credit,BR,Avast Brazil,1883448942,DR_BRAZIL-ENTITY,Declined,DR_BRAZIL-ENTITY,99,29.018051,BRL,MasterCard,netgiro-br,3,99,29.018051,BRL,46969296500,27001,Transaction refused[62] [null] [] [],None,New,13259159000,,2018-04-18 06:02:11,2018-04-18 01:02:11,01,6681601083,Authorize,0,2018-04-08 01:03:36,8,01,27001,Transaction refused[62] [null] [] [],txn_refused,Hard Fraud,10
2,20180418,avast,10001635101,8bc95f07-77b3-4bd4-abfa-5fee6658eb63,None,57E041B0B2F3D9A5,Metabank,US,435880,Visa,Visa Classic,Consumer,Debit Hybrid (PIN and Signature),0625,None,None,18,wednesday,None,Prepaid,US,DRI*AVAST Software,311009012882,DR_INC-ENTITY,Declined,DR_INC-ENTITY,53.99,53.99,USD,Visa,firstdata,3,53.99,53.99,USD,46969386300,402,12: INV TRAN,None,New,13259490100,,2018-04-18 06:03:02,2018-04-18 01:03:02,01,6681607843,Authorize,0,2018-04-03 01:10:44,3,01,402,12: INV TRAN,Base,Base,15
3,20180425,defendbr,10001895901,9f5ecc0d-2529-4930-9fc5-93daf88c9e74,3,899EBF821D876833,ITAU UNIBANCO S.A.,BR,521397,MasterCard,Platinum MasterCard Card,Consumer,True credit (No PIN/Signature capability),0620,None,None,25,wednesday,None,Credit,BR,DRI*BitDefender,75FA81DE-5AC5-5DCF-E97C-8F80A08ADE1E,DR_BRAZIL-ENTITY,Declined,DR_BRAZIL-ENTITY,229.99,66.742732,BRL,MasterCard,pagador,3,229.99,66.742732,BRL,47202743400,530,7: Do Not Honor,None,New,13259627500,,2018-04-25 06:01:35,2018-04-25 01:01:35,01,6706709733,Authorize,0,2018-04-16 01:01:42,16,01,530,7: Do Not Honor,do_not_honor,do_not_honor,9
4,20180418,avast,10001966301,1bf185fb-88cd-457c-85a2-a697021fc538,None,0FDA420D7B7FDBDC,"Chase Bank USA, National Association",US,424631,Visa,Visa Business,Business,True credit (No PIN/Signature capability),0718,None,None,18,wednesday,None,Credit,US,DRI*AVAST Software,311009012882,DR_INC-ENTITY,Declined,DR_INC-ENTITY,64.94,64.94,USD,Visa,firstdata,3,64.94,64.94,USD,46969539100,201,14: INV ACCT NUM,None,New,13259788300,,2018-04-18 06:05:07,2018-04-18 01:05:07,01,6681621783,Authorize,0,2018-04-03 01:12:21,3,01,201,14: INV ACCT NUM,Base,Base,15
5,20180519,adbehru,10002261401,0d1fbb95-019b-4f5f-8c53-f96ae78c2d0a,None,NON3DS,SBERBANK of Russia,RU,427609,Visa,Visa Classic,Consumer,NON USA Consumer Debit - No Pin Access,1018,RETRY_DECLINED.2,None,19,saturday,None,Debit,RU,DRI*Adobe Systems,1411163460-R

In [5]:
'''Queries all successful transactions to generate bin profile'''
import time

import pandas as pd


from pyspark.sql.types import *
from pyspark.sql.functions import col, count, rand, collect_list, explode, struct, count, lit, lower
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.functions import udf
from pyspark.sql.types import *
import pyspark_cassandra
        
import calendar, datetime
from datetime import timedelta

WORK_DIR = '/var/spark/ml_files/'


def convert_status(value):
    if value == 'Completed' or value == 'Reversed':
        return 1
    else:
        return 0
    
    
def cast_to_int(value, default=-1):
    try:
        return int(value)
    except (ValueError, TypeError):
        return default


#Convert from str to datetime
def to_date(datestr):
    struct = time.strptime(datestr, "%Y-%m-%d %H:%M:%S")
    date = datetime.date(struct.tm_year,struct.tm_mon,struct.tm_mday)
    return date

#Get difference between d2 and d1 in days.
def days_between(d1, d2):
    d1 = to_date(d1)
    d2 = to_date(d2)
    return abs((d2 - d1).days)

#Determines the week (number) of the month
def week_of_month(datestr):
    date = to_date(datestr)
    #Calendar object. 6 = Start on Sunday, 0 = Start on Monday
    cal_object = calendar.Calendar(6)
    month_calendar_dates = cal_object.itermonthdates(date.year,date.month)

    day_of_week = 1
    week_number = 1

    for day in month_calendar_dates:
        #add a week and reset day of week
        if day_of_week > 7:
            week_number += 1
            day_of_week = 1

        if date == day:
            break
        else:
            day_of_week += 1

    return str(week_number)

def to_weekday(datestr):
    struct = time.strptime(datestr, "%Y-%m-%d %H:%M:%S")
    date = datetime.date(struct.tm_year,struct.tm_mon,struct.tm_mday)
    return date.isoweekday()

def daterange(date1_str, date2_str):
    start_date = to_date(date1_str + ' 00:00:00')
    end_date = to_date(date2_str + ' 00:00:00')
    if start_date > end_date:
        raise ValueError('start date cannot be after the end date.')
    for n in range(int((end_date - start_date).days) + 1):
        yield str(start_date + timedelta(n))
            
def date_range_condition_values(start_date, end_date):
    dates = daterange(start_date, end_date)
    for idx, date in enumerate(dates):
            yield date

#Convert from str to datetime
def to_date(datestr):
    struct = time.strptime(datestr, "%Y-%m-%d %H:%M:%S")
    date = datetime.date(struct.tm_year,struct.tm_mon,struct.tm_mday)
    return date


    
udfValueToStatus = udf(convert_status, IntegerType())  

createDDL = """CREATE OR REPLACE TEMPORARY VIEW cpg_transaction_history
     USING org.apache.spark.sql.cassandra
     OPTIONS (
     table "cpg_transaction_history",
     keyspace "sarsks",
     cluster "MLP Cluster",
     pushdown "true")"""
spark.sql(createDDL)

# df = spark.sql("SELECT * FROM cpg_transaction_history WHERE received_date like '2018-11-%' or received_date like '2018-12-%' ")
df = spark.sql("SELECT bin, bank_code, payment_amount_usd, subscription_id, received_date FROM cpg_transaction_history WHERE received_date like '2018-01-%' or received_date like '2018-02-%' or received_date like '2018-03-%' or received_date like '2018-04-%' or received_date like '2018-05-%' or received_date like '2018-06-%' or received_date like '2018-07-%' or received_date like '2018-08-%' or received_date like '2018-09-%' or received_date like '2018-10-%' or received_date like '2018-11-%' or received_date like '2018-12-%' \
     or received_date like '2019-01-%' or received_date like '2019-02-%' or received_date like '2019-03-%' ")

sdf =  df\
    .filter("new_status in ('Completed')")\
    .cache()

print(sdf.schema)
                                            
                            
time_start = time.time()

def _map_to_pandas(rdds):
    """ Needs to be here due to pickling issues """
    return [pd.DataFrame(list(rdds))]

def toPandas(df, n_partitions=None):
    """
    Returns the contents of `df` as a local `pandas.DataFrame` in a speedy fashion. The DataFrame is
    repartitioned if `n_partitions` is passed.
    :param df:              pyspark.sql.DataFrame
    :param n_partitions:    int or None
    :return:                pandas.DataFrame
    """
    if n_partitions is not None: df = df.repartition(n_partitions)
    df_pand = df.rdd.mapPartitions(_map_to_pandas).collect()
    df_pand = pd.concat(df_pand)
    df_pand.columns = df.columns
    return df_pand

# sdf_filtered = sdf
#         .filter("event_uuid != ''")  #.toPandas()

# pdf_all = toPandas(sdf_filtered, 6)  
successful_txn = sdf.toPandas()
# successful_txn_11_12 = sdf.toPandas()
print("# total time:", time.time() - time_start)      # total time: 2364.714904308319
    
successful_txn.shape
# successful_txn_11_12.shape

StructType(List(StructField(bin,StringType,true),StructField(bank_code,StringType,true),StructField(payment_amount_usd,StringType,true),StructField(subscription_id,StringType,true),StructField(received_date,DateType,true)))
# total time: 93.047771692276


(5250412, 5)

In [10]:
import pandas as pd
WORK_DIR = '/var/spark/ml_files/'
successful_txn = pd.read_csv(WORK_DIR + 'successful_txn_2018_01_to_2019_03.csv')
successful_txn.shape

(18209867, 5)

In [11]:
successful_txn['payment_amount_usd'] = successful_txn['payment_amount_usd'].astype(float)


In [12]:
grouped = successful_txn['payment_amount_usd'].groupby(successful_txn['bank_code']).agg([('Mean','mean'), ('Median','median'),('Min','min'),('Max','max'),('StdDev','std'), ('Max_95', lambda x : x.quantile(.95)), ('Max_99', lambda x : x.quantile(.99)), ('Min_1', lambda x : x.quantile(.1)), ('Min_10', lambda x : x.quantile(.10)) , 'count']).reset_index()

bank_code_profile = grouped[grouped['count'] >= 3]
bank_code_profile.to_csv(WORK_DIR + 'bank_code_profile.csv')
bank_code_profile


,bank_code,Mean,Median,Min,Max,StdDev,Max_95,Max_99,Min_1,Min_10,count
0,,62.277421,59.990000,2.990000,199.541097,29.824982,105.727000,166.147303,22.902000,22.902000,479
1,00016AB58F593ED5,46.342498,46.509947,2.270000,169.990000,32.951066,104.390000,136.014000,8.965130,8.965130,261
2,0005BCE483B4DC42,48.086858,47.500000,2.270000,193.260000,36.714324,112.198000,153.358000,8.949047,8.949047,213
6,0014678C3CCF1020,52.962367,53.075000,2.270000,192.590000,33.007587,106.907500,164.857200,10.060000,10.060000,168
9,001882F399A66C00,40.996332,33.940000,1.940000,171.010000,32.506896,91.241000,150.140000,5.162000,5.162000,242
11,001D0F8ACA086BDA,56.308154,57.078582,2.260000,326.445271,39.003901,108.486509,175.336000,9.990000,9.990000,191
16,0026CCD52F9C05B0,45.743890,46.728414,2.490000,372.360000,36.624447,88.199000,118.947773,9.291948,9.291948,203
19,002CFCDD6ACE30D0,50.078118,44.990000,2.280000,1044.000000,77.001804,104.762000,222.803600,4.990000,4.990000,223
23,0031B608F3566AC4,45.490860,49.045000,2.490000,195.510000,34.933468,90.673352,150.890000,7.340000,7.340000,192
31,003F47ECE4256177,47.318685,49.990000,2.270000,169.990000,34.801363,97.454500,163.055000,9.301660,9.301660,174


In [12]:
# successful_txn = pd.read_csv(WORK_DIR + 'successful_txn_01_to_10.csv')
# successful_txn_2019 = pd.read_csv(WORK_DIR + 'successful_txn_2019_01_to_03.csv')
# successful_txn.shape

/opt/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (6,15,21,39) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(11653822, 5)

In [14]:
# fields = ['bin', 'payment_amount_usd', 'subscription_id', 'received_date']

# successful_txn = pd.concat([successful_txn[fields], successful_txn_11_12[fields], successful_txn_2019[fields]], ignore_index=True)
# successful_txn.shape  #16904223

TypeError: unsupported operand type(s) for -: 'tuple' and 'int'

In [9]:
import datetime
from datetime import timedelta

def to_date(datestr):
    """Convert from date string to date"""
    try:
        if len(datestr) <= 10:
            struct = time.strptime(datestr, "%Y-%m-%d")
        else:
            struct = time.strptime(datestr, "%Y-%m-%d %H:%M:%S")
        date = datetime.date(struct.tm_year, struct.tm_mon, struct.tm_mday)
    except Exception as ex:
        raise ValueError("date '{}' does not match format 'yyyy-mm-dd'".format(datestr))
    return date

def to_month(datestr):
    """Converts date string to month, e.g: '2018-05-02 00:00:00' to be '5' """
    return to_date(datestr).month

def to_year_month(datestr):
    """Converts date string to month, e.g: '2018-05-02 00:00:00' to be '5' """
    _date = to_date(datestr)
    return "{}-{}".format(_date.year, _date.month)

In [60]:
successful_txn['received_date'].astype(str).value_counts().sort_index().to_csv(WORK_DIR + 'successful_txn_group_by_date_2018_2019_03.csv')

In [17]:
successful_txn['bin'].fillna('', inplace=True)

In [20]:
FEATURES_FLOAT = ['bin']
def handle_feat_float(df):
        for feat in FEATURES_FLOAT:
            if feat in FEATURES_FLOAT:
                df[feat] = df[feat].fillna('').apply(str).str.replace('.0', '', regex=False)
        return df

handle_feat_float(successful_txn)    

,bin,payment_amount_usd,subscription_id,received_date
0,494000,95.590000,2448337709,2018-02-08
1,544902,21.240000,2476420509,2018-02-08
2,376066,22.000000,3197717709,2018-02-08
3,453997,20.730000,3197949709,2018-02-08
4,450550,15.968703,3198114509,2018-02-08
5,373239,20.000000,3198929909,2018-02-08
6,420767,18.500000,3198955009,2018-02-08
7,372719,85.520000,3199000209,2018-02-08
8,371557,16.990000,3199119809,2018-02-08
9,426354,22.812962,3199280709,2018-02-08


In [8]:
successful_txn['bin'].value_counts()

414720    165635
424631    142119
601100    142061
440066    133179
532610    125374
426684    102980
542418    100923
546616     96799
517805     92373
414709     90112
480213     70277
526471     67004
527289     58569
451842     57994
427638     56429
450979     54698
434256     54567
438857     53353
410039     50558
462845     50448
515590     50062
544731     49095
446542     48809
490172     47635
510737     46559
414740     46530
436420     43186
422061     42486
492181     41821
415417     41786
           ...  
533840         1
464885         1
487357         1
484397         1
558895         1
402728         1
442608         1
418460         1
427242         1
364734         1
460141         1
416799         1
511838         1
411502         1
493885         1
412121         1
550455         1
522394         1
515988         1
552172         1
551967         1
424326         1
526177         1
371914         1
456744         1
374955         1
404222         1
467470        

In [20]:
grouped = successful_txn['payment_amount_usd'].groupby(successful_txn['bin']).agg([('Mean','mean'), ('Median','median'),('Min','min'),('Max','max'),('StdDev','std'), ('Max_95', lambda x : x.quantile(.95)), ('Max_99', lambda x : x.quantile(.99)), ('Min_1', lambda x : x.quantile(.1)), ('Min_10', lambda x : x.quantile(.10)) , 'count']).reset_index()

bin_profile = grouped[grouped['count'] >= 3]
bin_profile
bin_profile.to_csv(WORK_DIR + 'bin_profile.csv')
bin_profile

,bin,Mean,Median,Min,Max,StdDev,Max_95,Max_99,Min_1,Min_10,count
0,230540,23.170809,25.550000,6.990000,52.900000,12.315995,45.725000,50.275000,9.100000,9.100000,126
2,352811,37.533333,40.000000,32.000000,40.600000,4.801389,40.540000,40.588000,33.600000,33.600000,3
5,352815,172.055652,53.700000,18.000000,562.822610,261.654900,489.709218,548.199932,22.200000,22.200000,4
6,352818,15.643014,4.944068,4.800000,51.479167,16.386331,40.881000,49.359534,4.800000,4.800000,12
7,352820,39.909110,32.177332,30.700000,73.000000,16.556696,64.725000,71.345000,31.100000,31.100000,6
8,352821,49.015144,49.159598,48.102689,50.518630,0.988569,50.246824,50.464269,48.115695,48.115695,5
9,352822,35.688928,39.518769,18.714051,49.360468,9.785019,49.219043,49.332183,21.110764,21.110764,13
10,352823,41.450000,30.900000,30.900000,73.100000,21.100000,66.770000,71.834000,30.900000,30.900000,4
11,352824,56.229756,55.059513,40.000000,74.800000,14.297354,71.997854,74.239571,44.200000,44.200000,4
12,352830,38.264198,40.300000,30.800000,41.656793,5.019725,41.483274,41.622089,33.590000,33.590000,4


In [22]:
grouped = successful_txn['payment_amount_usd'].groupby(successful_txn['bin']).agg([('Mean','mean'), ('Median','median'),('Min','min'),('Max','max'),('StdDev','std'), ('Max_95', lambda x : x.quantile(.95)), ('Max_99', lambda x : x.quantile(.99)), ('Min_1', lambda x : x.quantile(.1)), ('Min_10', lambda x : x.quantile(.10)) , 'count']).reset_index()

bin_profile = grouped[grouped['count'] >= 3]
bin_profile
bin_profile.to_csv(WORK_DIR + 'bin_profile.csv')
bin_profile

,bin,Mean,Min,Max,StdDev,Max_95,Max_99,Min_1,Min_10,count
0,,60.786311,2.180000,228.240000,26.148084,99.990000,129.134000,23.928000,23.928000,553
1,230540,23.170809,6.990000,52.900000,12.315995,45.725000,50.275000,9.100000,9.100000,126
2,352810,18.364580,18.364580,18.364580,NaN,18.364580,18.364580,18.364580,18.364580,1
3,352811,37.533333,32.000000,40.600000,4.801389,40.540000,40.588000,33.600000,33.600000,3
4,352812,30.898007,30.898007,30.898007,NaN,30.898007,30.898007,30.898007,30.898007,1
5,352814,31.400000,31.400000,31.400000,NaN,31.400000,31.400000,31.400000,31.400000,1
6,352815,172.055652,18.000000,562.822610,261.654900,489.709218,548.199932,22.200000,22.200000,4
7,352818,15.643014,4.800000,51.479167,16.386331,40.881000,49.359534,4.800000,4.800000,12
8,352820,39.909111,30.700000,73.000000,16.556696,64.725000,71.345000,31.100000,31.100000,6
9,352821,49.015144,48.102689,50.518630,0.988569,50.246824,50.464269,48.115695,48.115695,5


In [24]:
bin_profile.shape #(32272, 10)  33436

(38194, 10)

In [25]:
bin_profile.to_csv(WORK_DIR + 'bin_profile_2018_01_to_2019_03.csv')

In [5]:
'''Queries all raw  transactions data'''
import time

import pandas as pd


from pyspark.sql.types import *
from pyspark.sql.functions import col, count, rand, collect_list, explode, struct, count, lit, lower
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.functions import udf
from pyspark.sql.types import *
import pyspark_cassandra
        
import calendar, datetime
from datetime import timedelta

WORK_DIR = '/var/spark/ml_files/'


def convert_status(value):
    if value == 'Completed' or value == 'Reversed':
        return 1
    else:
        return 0
    
    
def cast_to_int(value, default=-1):
    try:
        return int(value)
    except (ValueError, TypeError):
        return default


#Convert from str to datetime
def to_date(datestr):
    struct = time.strptime(datestr, "%Y-%m-%d %H:%M:%S")
    date = datetime.date(struct.tm_year,struct.tm_mon,struct.tm_mday)
    return date

#Get difference between d2 and d1 in days.
def days_between(d1, d2):
    d1 = to_date(d1)
    d2 = to_date(d2)
    return abs((d2 - d1).days)

#Determines the week (number) of the month
def week_of_month(datestr):
    date = to_date(datestr)
    #Calendar object. 6 = Start on Sunday, 0 = Start on Monday
    cal_object = calendar.Calendar(6)
    month_calendar_dates = cal_object.itermonthdates(date.year,date.month)

    day_of_week = 1
    week_number = 1

    for day in month_calendar_dates:
        #add a week and reset day of week
        if day_of_week > 7:
            week_number += 1
            day_of_week = 1

        if date == day:
            break
        else:
            day_of_week += 1

    return str(week_number)

def to_weekday(datestr):
    struct = time.strptime(datestr, "%Y-%m-%d %H:%M:%S")
    date = datetime.date(struct.tm_year,struct.tm_mon,struct.tm_mday)
    return date.isoweekday()

def daterange(date1_str, date2_str):
    start_date = to_date(date1_str + ' 00:00:00')
    end_date = to_date(date2_str + ' 00:00:00')
    if start_date > end_date:
        raise ValueError('start date cannot be after the end date.')
    for n in range(int((end_date - start_date).days) + 1):
        yield str(start_date + timedelta(n))
            
def date_range_condition_values(start_date, end_date):
    dates = daterange(start_date, end_date)
    for idx, date in enumerate(dates):
            yield date

#Convert from str to datetime
def to_date(datestr):
    struct = time.strptime(datestr, "%Y-%m-%d %H:%M:%S")
    date = datetime.date(struct.tm_year,struct.tm_mon,struct.tm_mday)
    return date


    
udfValueToStatus = udf(convert_status, IntegerType())  

createDDL = """CREATE OR REPLACE TEMPORARY VIEW cpg_transaction_history
     USING org.apache.spark.sql.cassandra
     OPTIONS (
     table "cpg_transaction_history",
     keyspace "sarsks",
     cluster "MLP Cluster",
     pushdown "true")"""
spark.sql(createDDL)

df = spark.sql("SELECT * FROM cpg_transaction_history WHERE received_date like '2019-01-%' or received_date like '2019-02-%' or received_date like '2019-03-%' ")
# df = spark.sql("SELECT * FROM cpg_transaction_history WHERE received_date like '2018-01-%' or received_date like '2018-02-%' or received_date like '2018-03-%' or received_date like '2018-04-%' or received_date like '2018-05-%' or received_date like '2018-06-%' or received_date like '2018-07-%' or received_date like '2018-08-%' or received_date like '2018-09-%' or received_date like '2018-10-%' or received_date like '2018-11-%' or received_date like '2018-12-%' ")
# df = spark.sql("SELECT * FROM cpg_transaction_history WHERE received_date like '2018-09-%' or received_date like '2018-10-%' or received_date like '2018-11-%' or received_date like '2018-12-%' ")

sdf =  df\
    .withColumn("success", udfValueToStatus("new_status"))\
    .cache()

print(sdf.schema)
    

                                             
                            
time_start = time.time()

def _map_to_pandas(rdds):
    """ Needs to be here due to pickling issues """
    return [pd.DataFrame(list(rdds))]

def toPandas(df, n_partitions=None):
    """
    Returns the contents of `df` as a local `pandas.DataFrame` in a speedy fashion. The DataFrame is
    repartitioned if `n_partitions` is passed.
    :param df:              pyspark.sql.DataFrame
    :param n_partitions:    int or None
    :return:                pandas.DataFrame
    """
    if n_partitions is not None: df = df.repartition(n_partitions)
    df_pand = df.rdd.mapPartitions(_map_to_pandas).collect()
    df_pand = pd.concat(df_pand)
    df_pand.columns = df.columns
    return df_pand

# sdf_filtered = sdf
#         .filter("event_uuid != ''")  #.toPandas()

# pdf_all = toPandas(sdf, 64)  
pdf_all = sdf.toPandas()
print("# total time:", time.time() - time_start)      # total time: 2364.714904308319
    
pdf_all.shape

StructType(List(StructField(received_date,DateType,true),StructField(site_id,StringType,true),StructField(subscription_id,StringType,true),StructField(transaction_id,StringType,true),StructField(transaction_date,TimestampType,true),StructField(added_expiry_years,StringType,true),StructField(bank_code,StringType,true),StructField(bank_name,StringType,true),StructField(billing_country,StringType,true),StructField(bin,StringType,true),StructField(card_brand,StringType,true),StructField(card_category,StringType,true),StructField(card_class,StringType,true),StructField(card_usage,StringType,true),StructField(cc_expiration_date,StringType,true),StructField(cid,StringType,true),StructField(date_increment,StringType,true),StructField(day_of_month,StringType,true),StructField(day_of_week,StringType,true),StructField(event_uuid,StringType,true),StructField(exp_setting_id,StringType,true),StructField(funding_source,StringType,true),StructField(issuer_country,StringType,true),StructField(merchant_

(8682889, 46)

In [6]:
pdf_all['received_date'].value_counts()

2019-03-28    155108
2019-01-28    127637
2019-01-02    121899
2019-02-28    115943
2019-03-24    111551
2019-01-10    110560
2019-01-24    108504
2019-01-12    107895
2019-03-21    107513
2019-01-17    107166
2019-02-01    105650
2019-01-18    105255
2019-01-16    105209
2019-02-04    104942
2019-01-22    104742
2019-03-14    104610
2019-01-25    104479
2019-01-04    104044
2019-02-09    103816
2019-02-27    103767
2019-02-02    103247
2019-01-03    103116
2019-02-07    103012
2019-02-08    102904
2019-01-05    102689
2019-03-01    102613
2019-02-06    102545
2019-01-20    102338
2019-03-07    102291
2019-02-05    101375
               ...  
2019-03-02     91754
2019-02-25     91700
2019-03-10     90222
2019-03-13     90213
2019-02-20     90042
2019-03-17     89880
2019-01-29     89161
2019-03-11     89075
2019-03-05     88477
2019-02-26     87241
2019-02-21     87132
2019-02-12     86901
2019-02-18     86500
2019-02-15     85158
2019-02-16     84134
2019-02-22     83701
2019-02-11   

In [7]:
pdf_all.to_csv(WORK_DIR + 'raw_txns_2019_01_to_03.csv')

In [12]:
pdf_all_07[pdf_all_07['subsegment_id'].isin(['16220370500','16203155800'])].sort_values(by=['subsegment_id', 'transaction_date'])

,Unnamed: 0,received_date,site_id,subscription_id,transaction_id,transaction_date,added_expiry_years,bank_code,bank_name,billing_country,...,request_currency,requisition_id,response_code,response_message,status,subsegment_id,transaction_date_in_string,transaction_hour,transaction_type,success
849598,33205,2018-07-04,adbehru,11078418901,7005906373,2018-07-04 07:18:31,NaN,NON3DS,QIWI Bank (JSC),RU,...,RUB,49596344600,0,SUCCESS[00] [5RXJY6] [] [],New,16203155800,2018-07-04 02:18:31,2,Authorize,1.0
1246825,2883,2018-07-11,adbehru,11078418901,7039424523,2018-07-11 07:07:15,NaN,NON3DS,QIWI Bank (JSC),RU,...,RUB,49813562200,0,SUCCESS[00] [5V3T7M] [] [],New,16203155800,2018-07-11 02:07:15,2,Authorize,1.0
1268958,25016,2018-07-13,adbehru,11078418901,7050286213,2018-07-13 07:23:10,NaN,NON3DS,QIWI Bank (JSC),RU,...,RUB,49876637000,100,reversal SUCCESS,Completed,16203155800,2018-07-13 02:23:10,2,Authorize,0.0
1316867,34061,2018-07-06,adbehru,10441599501,7016103103,2018-07-06 06:50:49,NaN,NON3DS,"""YANDEX.MONEY"" NBCO LLC",RU,...,RUB,49658579900,0,SUCCESS[00] [862273] [] [],New,16220370500,2018-07-06 01:50:49,1,Authorize,1.0
1053051,3443,2018-07-13,adbehru,10441599501,7050276173,2018-07-13 07:20:50,NaN,NON3DS,"""YANDEX.MONEY"" NBCO LLC",RU,...,RUB,49876337400,100,reversal SUCCESS,Completed,16220370500,2018-07-13 02:20:50,2,Authorize,0.0
2420044,9907,2018-07-15,adbehru,10441599501,7059935623,2018-07-15 06:46:52,NaN,NON3DS,"""YANDEX.MONEY"" NBCO LLC",RU,...,RUB,49938868600,0,SUCCESS[00] [959322] [] [],New,16220370500,2018-07-15 01:46:52,1,Authorize,1.0


In [32]:
# pdf_all_07[pdf_all_07['subsegment_id'] == 13865645800]
pdf_all_07[(pdf_all_07['success'] == 1) & (pdf_all_07['renew_att_num'] > 1) & (pdf_all_07['cid'].notna())]

,Unnamed: 0,received_date,site_id,subscription_id,transaction_id,transaction_date,added_expiry_years,bank_code,bank_name,billing_country,bin,card_brand,card_category,card_class,card_usage,cc_expiration_date,cid,date_increment,day_of_month,day_of_week,event_uuid,exp_setting_id,funding_source,issuer_country,merchant_descriptor,merchant_number,mid_entity_code,new_status,order_entity_code,payment_amount,payment_amount_usd,payment_currency,payment_method_id,payment_service_id,renew_att_num,request_amount,request_amount_usd,request_currency,requisition_id,response_code,response_message,status,subsegment_id,transaction_date_in_string,transaction_hour,transaction_type,success
618,618,2018-07-25,avast,10385286901,7109748483,2018-07-25 06:31:10,NaN,NON3DS,Yapi Ve Kredi Bankasi A.S.,TR,491205.0,Visa,Visa Classic,Consumer,True credit (No PIN/Signature capability),0219,RETRY_DECLINED.2,NaN,25,Wednesday,ea91f9e0-4af1-46d0-8e3c-df088d4990ce,NaN,Credit,TR,DRI*AVAST Software,1156962062,DR_IRELAND-ENTITY,Completed,DR_IRELAND-ENTITY,94.39,94.390000,USD,Visa,netgiro-seb,2.0,94.39,94.390000,USD,50252909600,0,SUCCESS[00] [746557] [] [],New,13900925300,2018-07-25 01:31:10,1,Authorize,1.0
1248,1248,2018-07-31,tmamer,2143848009,7139109843,2018-07-31 07:19:36,0.0,6DAB740CF8E74355,Charter Oak Federal Credit Union,US,411251.0,Visa,Visa Traditional Rewards,Consumer,True credit (No PIN/Signature capability),0418,RETRY_DECLINED.2,0.0,31,Tuesday,1f86b34f-292a-4ec4-b76f-9375b9e4931b,3892981.0,Credit,US,DRI*Trend Micro,7236114,DR_INC-ENTITY,Completed,DR_INC-ENTITY,81.76,81.760000,USD,Visa,litle,3.0,81.76,81.760000,USD,50445217500,100,000: Approved,New,13812476100,2018-07-31 02:19:36,2,Authorize,1.0
2184,2184,2018-07-30,avgstore,11974814201,7135121403,2018-07-30 09:46:33,0.0,F573CC091A707D81,Navy Federal Credit Union,US,403216.0,Visa,Visa Traditional,Consumer,True credit (No PIN/Signature capability),0817,RETRY_DECLINED.2,0.0,30,Monday,6de9dfeb-754c-4f79-a107-4754cfd1da9f,3916951.0,Credit,US,DRI*AVG Technologies,7236114,DR_INC-ENTITY,Completed,DR_INC-ENTITY,64.99,64.990000,USD,Visa,litle,3.0,64.99,64.990000,USD,50424355600,100,000: Approved,New,16747757500,2018-07-30 04:46:33,4,Authorize,1.0
4038,4038,2018-07-29,avgstore,11970853601,7131129423,2018-07-29 07:00:30,0.0,D62FFBC318D7C238,"First Tennessee Bank, National Association",US,480706.0,Visa,Visa Purchasing with Fleet,Purchase,True credit (No PIN/Signature capability),0517,RETRY_DECLINED.2,0.0,29,Sunday,774e60db-1f0d-4fcd-98ff-ebd61fcda952,3916951.0,Credit,US,DRI*AVG Technologies,7236114,DR_INC-ENTITY,Completed,DR_INC-ENTITY,83.99,83.990000,USD,Visa,litle,3.0,83.99,83.990000,USD,50387472900,100,000: Approved,New,16738489500,2018-07-29 02:00:30,2,Authorize,1.0
7428,7428,2018-07-26,avast,3471099101,7118353193,2018-07-26 18:15:46,NaN,13C5143010ACCE29,NaN,US,372879.0,American Express,NaN,NaN,True credit (No PIN/Signature capability),0620,RETRY_DECLINED.2,NaN,26,Thursday,7bafa3b7-b266-4a50-a739-92e73d95b122,NaN,NaN,NaN,DRI*AVAST Software,311009012882,DR_INC-ENTITY,Completed,DR_INC-ENTITY,49.99,49.990000,USD,AmericanExpress,firstdata,2.0,49.99,49.990000,USD,50311367900,100,00: Approved,New,13798605400,2018-07-26 13:15:46,13,Authorize,1.0
8763,8763,2018-07-08,avgstore,10294046601,7024206113,2018-07-08 06:28:44,NaN,NON3DS,BLOM BANK SAL,LB,524334.0,MasterCard,Titanium MasterCard Card,Consumer,True credit (No PIN/Signature capability),0919,RETRY_FAILED.2,NaN,8,Sunday,8c719a4e-628f-42ad-a2d6-a8a6dc2b42ef,NaN,Credit,LB,DRI*AVG Technologies,1156962062,DR_IRELAND-ENTITY,Completed,DR_IRELAND-ENTITY,44.99,44.990000,USD,MasterCard,netgiro-seb,3.0,44.99,44.990000,USD,49718141100,0,SUCCESS[00] [091104] [] [],New,13750250800,2018-07-08 01:28:44,1,Authorize,1.0
15339,15339,2018-07-11,cppro,10743893401,7039383923,2018-07-11 06:58:56,NaN,NON3DS,MBANK S.A.,PL,547890.0,MasterCard,MasterCard Corporate Immediate Debi,Business,True credit (No PIN/Signature capability),0421,RETRY_DECLINED.2,NaN,11,Wednesday,233e650e-9d18-4f89-960f-32c94

In [56]:
pdf_all_07[pdf_all_07['subsegment_id'] == 16460118700].sort_values(by=['transaction_date','cid'], na_position='first')[['renew_att_num', 'event_uuid', 'transaction_date', 'cid', 'success']]
# pdf_all_07[pdf_all_07['subscription_id'] == '9171379701']

,renew_att_num,event_uuid,transaction_date,cid,success
948504,1.0,f4188263-04fc-4695-b306-12aeb1f58b91,2018-07-11 07:36:50,NaN,0.0
2141335,1.0,96013982-5e77-4e3f-8d95-9423d4915fa7,2018-07-11 07:36:51,RETRY_DECLINED.2,0.0
1222562,2.0,f937e46c-a646-413e-bbb0-fc49b1c621e3,2018-07-18 07:28:04,NaN,0.0
1828997,2.0,1276b843-35f7-4468-8050-606a0da7ec34,2018-07-18 07:28:05,RETRY_DECLINED.2,0.0
14,3.0,7021a0e1-8897-4055-9cb6-be6ed666984a,2018-07-25 07:27:32,NaN,0.0
671473,3.0,7cdad381-12bc-4d91-b5a0-e29c0322c160,2018-07-25 07:27:33,RETRY_DECLINED.2,0.0


In [49]:
# pdf_all_07[pdf_all_07['subsegment_id'].isin([13865645800, 16460118700, 11815740300])].sort_values(by=['transaction_date'])
from IPython.display import display
pd.options.display.max_columns = None
pdf_all_07[pdf_all_07['subsegment_id'].isin([16738489500])].sort_values(by=['transaction_date'])[['renew_att_num','transaction_date','transaction_date_in_string']]

,renew_att_num,transaction_date,transaction_date_in_string
1162481,1.0,2018-07-21 11:15:27,2018-07-21 06:15:27
1745915,1.0,2018-07-21 11:15:27,2018-07-21 06:15:27
2473160,1.0,2018-07-21 11:15:27,2018-07-21 06:15:27
671974,2.0,2018-07-25 19:15:30,2018-07-25 14:15:30
182745,2.0,2018-07-25 19:15:31,2018-07-25 14:15:31
1077060,2.0,2018-07-25 19:15:31,2018-07-25 14:15:31
4038,3.0,2018-07-29 07:00:30,2018-07-29 02:00:30
2374576,3.0,2018-07-29 07:00:30,2018-07-29 02:00:30


In [60]:
from IPython.display import display
pd.options.display.max_columns = None
# pdf_all_07[pdf_all_07['subsegment_id'].isin(['16220370500','16203155800'])].sort_values(by=['subsegment_id', 'transaction_date'])
pdf_all_07[pdf_all_07['subsegment_id'].isin(['16220370500'])].sort_values(by=['subsegment_id', 'transaction_date'])

,Unnamed: 0,received_date,site_id,subscription_id,transaction_id,transaction_date,added_expiry_years,bank_code,bank_name,billing_country,bin,card_brand,card_category,card_class,card_usage,cc_expiration_date,cid,date_increment,day_of_month,day_of_week,event_uuid,exp_setting_id,funding_source,issuer_country,merchant_descriptor,merchant_number,mid_entity_code,new_status,order_entity_code,payment_amount,payment_amount_usd,payment_currency,payment_method_id,payment_service_id,renew_att_num,request_amount,request_amount_usd,request_currency,requisition_id,response_code,response_message,status,subsegment_id,transaction_date_in_string,transaction_hour,transaction_type,success
1316867,34061,2018-07-06,adbehru,10441599501,7016103103,2018-07-06 06:50:49,NaN,NON3DS,"""YANDEX.MONEY"" NBCO LLC",RU,510621.0,MasterCard,MasterCard New World Immediate Debit,Consumer,NON USA Consumer Debit - No Pin Access,0718,NaN,NaN,6,Friday,db46e9f3-a49f-4ae0-a864-f68a3b42351f,NaN,Debit,RU,DRI*Adobe Systems,1156962062,DR_IRELAND-ENTITY,Completed,DR_IRELAND-ENTITY,759.92,12.060260,RUB,MasterCard,netgiro-seb,4.0,759.92,12.060260,RUB,49658579900,0,SUCCESS[00] [862273] [] [],New,16220370500,2018-07-06 01:50:49,1,Authorize,1.0
1053051,3443,2018-07-13,adbehru,10441599501,7050276173,2018-07-13 07:20:50,NaN,NON3DS,"""YANDEX.MONEY"" NBCO LLC",RU,NaN,MasterCard,MasterCard New World Immediate Debit,Consumer,NON USA Consumer Debit - No Pin Access,0718,NaN,NaN,13,Friday,be7b2c47-d36b-4f06-9477-d7465623953e,NaN,Debit,RU,DRI*Adobe Systems,1156962062-RUB-pacific,DR_IRELAND-ENTITY,Reversed,DR_IRELAND-ENTITY,759.92,12.233536,RUB,MasterCard,netgiro-seb,5.0,759.92,12.233536,RUB,49876337400,100,reversal SUCCESS,Completed,16220370500,2018-07-13 02:20:50,2,Authorize,0.0
2420044,9907,2018-07-15,adbehru,10441599501,7059935623,2018-07-15 06:46:52,NaN,NON3DS,"""YANDEX.MONEY"" NBCO LLC",RU,510621.0,MasterCard,MasterCard New World Immediate Debit,Consumer,NON USA Consumer Debit - No Pin Access,0718,NaN,NaN,15,Sunday,a41df80a-c2d8-4423-8ff0-cb0078f65972,NaN,Debit,RU,DRI*Adobe Systems,1156962062,DR_IRELAND-ENTITY,Completed,DR_IRELAND-ENTITY,759.92,12.206418,RUB,MasterCard,netgiro-seb,6.0,759.92,12.206418,RUB,49938868600,0,SUCCESS[00] [959322] [] [],New,16220370500,2018-07-15 01:46:52,1,Authorize,1.0


In [57]:
import time
import calendar, datetime

#Convert from str to datetime
def to_date(datestr):
    struct = time.strptime(datestr, "%Y-%m-%d %H:%M:%S")
    date = datetime.date(struct.tm_year,struct.tm_mon,struct.tm_mday)
    return date

#Get difference between d2 and d1 in days.
def days_between(d1, d2):
    d1 = to_date(d1)
    d2 = to_date(d2)
    return abs((d2 - d1).days)


pdf_all_07[pdf_all_07['subsegment_id'].isin([16460118700])].groupby(["subscription_id", "subsegment_id"]).apply(merge_by_next_calendar_attempt)[['event_uuid', 'failed_event_uuid', 'renew_att_num', 'received_date', 'success', 'failed_attempt_date', 'failed_response_code', 'failed_response_message', 'failed_decline_type', 'days_between']]

# pdf_all_07[pdf_all_07['subsegment_id'].isin([13865645800, 16460118700, 11815740300])].groupby(["subscription_id", "subsegment_id"]).apply(merge_by_next_calendar_attempt)[['event_uuid', 'failed_event_uuid', 'renew_att_num', 'received_date', 'success', 'failed_attempt_date', 'failed_response_code', 'failed_response_message', 'failed_decline_type', 'days_between']]
# pdf_all_07[pdf_all_07['subscription_id'] == '9171379701'].groupby(["subscription_id", "subsegment_id"]).apply(merge_by_sub)

[1222562, 1828997]
--------------Test#3-------------
[14, 671473]
--------------Test#3-------------
[1222562, 1828997]
--------------Test#3-------------
[14, 671473]
--------------Test#3-------------


event_uuid  \
subscription_id subsegment_id                                                 
11831972601     16460118700   1828997  1276b843-35f7-4468-8050-606a0da7ec34   
                              671473   7cdad381-12bc-4d91-b5a0-e29c0322c160   

                                                          failed_event_uuid  \
subscription_id subsegment_id                                                 
11831972601     16460118700   1828997  96013982-5e77-4e3f-8d95-9423d4915fa7   
                              671473   1276b843-35f7-4468-8050-606a0da7ec34   

                                       renew_att_num received_date  success  \
subscription_id subsegment_id                                                 
11831972601     16460118700   1828997            2.0    2018-07-18      0.0   
                              671473             3.0    2018-07-25      0.0   

                                       failed_attempt_date  \
subscription_id subsegment_id                                
11831972601     16460118700   1828997  2018-07-11 02:36:51   
                              671473   2018-07-18 02:28:05   

                                       failed_response_code  \
subscription_id subsegment_id                                 
11831972601     16460118700   1828997               27048.0   
                              671473                27048.0   

                                                             failed_response_message  \
subscription_id subsegment_id                                                          
11831972601     16460118700   1828997  Lost or stolen card.[41] [] [122800] [122800]   
                              671473   Lost or stolen card.[41] [] [111800] [111800]   

                                      failed_decline_type days_between  
subscription_id subsegment_id                                           
11831972601     16460118700   1828997                                0  
                              671473                                 0

In [1]:
# Test #3 Merge failed previous calendar attempt with the next calendar attempt. This need to be on different date.
# Instead of getting the last attempt on the next calendar, we tried to get the successful txn on the next calendar first (to accomodate reverse txn)
# If successful txn is not found then look for the last attempt.

def to_day(datestr):
    """Converts date string to day, e.g: '2018-05-02 00:00:00' to be '2' """
    return to_date(datestr).day

def is_day_of_month_correct(transaction_date, day_of_month):
    txn_date_day = to_day(str(transaction_date))
    return str(day_of_month) == txn_date_day


def processor_mid_changed(initial_row, merged_row):
    if initial_row['payment_service_id'] == merged_row['payment_service_id']:
        if initial_row['merchant_number'] == merged_row['merchant_number']:
            return "processor_and_mid_unchanged"
        else:
            return "processor_unchanged_mid_changed"
    else:
        if initial_row['merchant_number'] == merged_row['merchant_number']:
            return "processor_changed_mid_unchanged"
        else:
            return "processor_changed_mid_changed"

# Instead of merging 1.3 into 2.1, we merge 1.3 to the successful txn of the next calendar, if not found then merge with the last attempt of the next calendar.
def merge_by_next_calendar_attempt(group):

    
    #Assign default values to additonal columns
    group['failed_attempt_date'] = ''
    group['failed_day_of_month'] = 0
    group['failed_response_code'] = ''
    group['failed_response_message'] = ''
    group['failed_decline_type'] = ''
    group['days_between'] = 0
    group['failed_event_uuid'] = ''
    
    group['failed_cc_expiration_date'] = ''
    group['expiration_date_changed'] = False
    group['processor_mid_changed'] = '' 
    group['failed_cid'] = ''
    group['failed_payment_service_id'] = ''
    group['failed_merchant_number'] = '' 
    
    group['failed_bank_code'] = ''
    group['bank_code_changed'] = False
    
    group['failed_bin'] = ''
    group['bin_changed'] = False
    
    new_group = pd.DataFrame(columns = group.columns)
    
    sorted_group = group.sort_values(by=['transaction_date', 'cid'], na_position='first')
    sorted_index = sorted_group.index.tolist()
    successful_df = sorted_group[sorted_group['success'] == 1]
    if not successful_df.empty:
        first_succes_index = successful_df.index[0]
        sorted_group_index = sorted_index[:sorted_index.index(first_succes_index) +1]
        sorted_group = sorted_group[sorted_group.index.isin(sorted_group_index)]
    
    initial_att_num = sorted_group.head(1)['renew_att_num'].iloc[0]
    last_att_num = sorted_group.tail(1)['renew_att_num'].iloc[0]
    
    if not last_att_num or not initial_att_num:
        sorted_group['event_uuid'] = '' 
        return sorted_group
        
    for att_num in range(int(initial_att_num), int(last_att_num)):
        next_att_num = int(att_num) + 1
        initial_att_indexes = sorted_group[sorted_group['renew_att_num'] == str(att_num)].index.tolist()
        next_att_indexes = sorted_group[sorted_group['renew_att_num'] == str(next_att_num)].index.tolist()

#         initial_att_indexes = sorted_group[sorted_group['renew_att_num'] == (att_num)].index.tolist()
#         next_att_indexes = sorted_group[sorted_group['renew_att_num'] ==  (next_att_num)].index.tolist()
    
        if initial_att_indexes and next_att_indexes:
            last_initial_att_index = initial_att_indexes[-1]

            last_next_att_index = next_att_indexes[-1]        
    
            merged = sorted_group.loc[last_next_att_index].copy(deep=True)
            initial_row = sorted_group.loc[last_initial_att_index]    
            failed_attempt_date = initial_row['transaction_date_in_string']                   
            merged['failed_attempt_date'] = failed_attempt_date

            merged['failed_day_of_month'] = int(to_day(initial_row['transaction_date_in_string']))
            merged['day_of_month'] = int(to_day(merged['transaction_date_in_string']))
            merged['failed_response_code'] = initial_row['response_code']

            failed_response_msg = initial_row['response_message']
            merged['failed_response_message'] = failed_response_msg  

            merged['failed_decline_type'] = decline_type(initial_row['response_message']) 
            merged['days_between'] = days_between_ds(merged)
            merged['failed_event_uuid'] =  initial_row['event_uuid']  
            
###### Below are additional fields  ########
            merged['failed_cc_expiration_date'] = initial_row['cc_expiration_date']
#         merged['expiration_date_changed'] = str(initial_row['cc_expiration_date'] is not merged['cc_expiration_date'])

            initial_cc = initial_row['cc_expiration_date'].replace('/', '')
            merged_cc = merged['cc_expiration_date'].replace('/', '')
            merged['expiration_date_changed'] = (initial_cc != merged_cc)

            merged['processor_mid_changed'] = processor_mid_changed(initial_row, merged)
            
            merged['failed_cid'] = initial_row['cid']        
            merged['failed_payment_service_id'] = initial_row['payment_service_id']
            merged['failed_merchant_number'] = initial_row['merchant_number']
            
            merged['failed_bank_code'] = initial_row['bank_code']
            merged['bank_code_changed'] = merged['bank_code'] != merged['failed_bank_code']
            
            merged['failed_bin'] = initial_row['bin']
            merged['bin_changed'] = merged['bin'] != merged['failed_bin']
##############################           
            
            new_group = new_group.append(merged)    
            
             
    if new_group.empty:
        sorted_group['event_uuid'] = '' 
        return sorted_group
    
    return new_group

In [9]:
# Merge failed previous calendar attempt with the next calendar attempt. This need to be on different date.
import time

import pandas as pd


from pyspark.sql.types import *
from pyspark.sql.functions import col, count, rand, collect_list, explode, struct, count, lit, lower
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.functions import udf
from pyspark.sql.types import *
import pyspark_cassandra
        
import calendar, datetime
from datetime import timedelta
import itertools

WORK_DIR = '/var/spark/ml_files/'


def convert_status(value):
    if value == 'Completed':
        return 1
    else:
        return 0
    
    
def cast_to_int(value, default=-1):
    try:
        return int(value)
    except (ValueError, TypeError):
        return default


#Convert from str to datetime
def to_date(datestr):
    struct = time.strptime(datestr, "%Y-%m-%d %H:%M:%S")
    date = datetime.date(struct.tm_year,struct.tm_mon,struct.tm_mday)
    return date

#Get difference between d2 and d1 in days.
def days_between(d1, d2):
    d1 = to_date(d1)
    d2 = to_date(d2)
    return abs((d2 - d1).days)

#Determines the week (number) of the month
def week_of_month(datestr):
    date = to_date(datestr)
    #Calendar object. 6 = Start on Sunday, 0 = Start on Monday
    cal_object = calendar.Calendar(6)
    month_calendar_dates = cal_object.itermonthdates(date.year,date.month)

    day_of_week = 1
    week_number = 1

    for day in month_calendar_dates:
        #add a week and reset day of week
        if day_of_week > 7:
            week_number += 1
            day_of_week = 1

        if date == day:
            break
        else:
            day_of_week += 1

    return str(week_number)

def to_weekday(datestr):
    struct = time.strptime(datestr, "%Y-%m-%d %H:%M:%S")
    date = datetime.date(struct.tm_year,struct.tm_mon,struct.tm_mday)
    return date.isoweekday()

def daterange(date1_str, date2_str):
    start_date = to_date(date1_str + ' 00:00:00')
    end_date = to_date(date2_str + ' 00:00:00')
    if start_date > end_date:
        raise ValueError('start date cannot be after the end date.')
    for n in range(int((end_date - start_date).days) + 1):
        yield str(start_date + timedelta(n))
            
def date_range_condition_values(start_date, end_date):
    dates = daterange(start_date, end_date)
    for idx, date in enumerate(dates):
            yield date

df_decline_type = pd.read_csv(WORK_DIR + 'Decline_Type.csv')

msg_group = {  
             'do_not_honor' : 'do not honor', 
             'attempt_lower_amount' : 'lower amount',
            'Insufficient Funds' : 'insufficient',
            'correct_cc_retry' : 'correct card',
            'invalid_cc' : 'invalid card',
            'lost_stolen' : 'lost or stolen',
            'invalid_account' : 'invalid account',
            'do_not_try_merchant_review' : 'do not try again/merchant review',
            'expired_card' : 'expired',
            'pickup_card' : 'pick',
            'blocked_first_used' : 'blocked',
            'invalid_txn' : 'invalid trans',
            'restricted_card' : 'restricted',
            'not_permitted' : 'not permitted',
            'expired card' : 'expired card',
            'unable to determine format' : 'determine format',
            'system error' : 'system error',
            'no reply' : 'no reply',
             'no charge model found' : 'no charge model found',
             'issuer unavailable' : 'issuer unavailable',
             'litle http response code' : 'litle http response code',
             'ioexception' : 'ioexception',
             'invalid merchant' : 'invalid merchant',
             'international filtering':'international filtering',
             'corrupt input data':'corrupt input data',
             'server error' : 'server error',
             'acquirer error' : 'acquirer error',
             'transaction refused[30]':'transaction refused[30]',
             'transaction refused[002]':'transaction refused[002]',
             'txn_refused' : 'refuse',
             'declined non generic': 'declined non generic',
             'declined' : 'decline', 
             'transaction not allowed at terminal': 'transaction not allowed at terminal',
             'error validating xml data': 'error validating xml data',
             'communication problems': 'communication problems',
             'new account info': 'new account info',
             'unable to connect to gateway': 'unable to connect to gateway'
            }

def group_response_msg(msg):
    other = 'Base'
    if isinstance(msg, str) == False:
        return other
    
    msg_lower = msg.lower()
    for key, val in msg_group.items():
        if val in msg_lower:
            return key
        
    return other   


def decline_type(response_msg):
    '''Converts to decline_type based on the given response_msg'''
    dec_type = df_decline_type[df_decline_type['DECLINE_TEXT'] == response_msg]['DECLINE_TYPE']
    if dec_type.empty or dec_type.iloc[0] == 'Base' :
        return group_response_msg(response_msg)
    else:
        return dec_type.iloc[0]

# #Convert from str to datetime
# def to_date(datestr):
#     struct = time.strptime(datestr, "%Y-%m-%d %H:%M:%S")
#     date = datetime.date(struct.tm_year,struct.tm_mon,struct.tm_mday)
#     return date

#Get difference between d2 and d1 in days.
def days_between(d1, d2):
    print('d1: {}, d2: {}'.format(d1, d2))
    d1 = to_date(d1)
    d2 = to_date(d2)
    print('after days_between')
    return abs((d2 - d1).days)

#Get difference between d2 and d1 in days.
def days_between_ds(df):
    d1 = to_date(df['failed_attempt_date'])
    d2 = to_date(df['transaction_date_in_string'])
    return abs((d2 - d1).days)

    
udfValueToStatus = udf(convert_status, IntegerType())  
# udfValueToStatus = udf(convert_status, StringType())  

createDDL = """CREATE OR REPLACE TEMPORARY VIEW cpg_transaction_history
     USING org.apache.spark.sql.cassandra
     OPTIONS (
     table "cpg_transaction_history",
     keyspace "sarsks",
     cluster "MLP Cluster",
     pushdown "true")"""
spark.sql(createDDL)

df = spark.sql("SELECT * FROM cpg_transaction_history WHERE received_date like '2019-01-%' or received_date like '2019-02-%' or received_date like '2019-03-%' ")
# df = spark.sql("SELECT * FROM cpg_transaction_history WHERE received_date like '2018-12-%' or received_date like '2019-01-%' or received_date like '2019-02-%' or received_date like '2019-03-%' ")
# df = spark.sql("SELECT * FROM cpg_transaction_history WHERE received_date like '2018-01-%' or received_date like '2018-02-%' or received_date like '2018-03-%' or received_date like '2018-04-%' or received_date like '2018-05-%' or received_date like '2018-06-%' or received_date like '2018-07-%' or received_date like '2018-08-%' or received_date like '2018-09-%' or received_date like '2018-10-%' or received_date like '2018-11-%' or received_date like '2018-12-%' ")

sdf =  df\
    .filter("!(renew_att_num == '1' and new_status in ('Completed') and cid is null and status == 'New')")\
    .withColumn("success", udfValueToStatus("new_status"))\
    .withColumn("day_of_month", df["day_of_month"].cast(IntegerType()))\
    .withColumn("day_of_week", lower(col("day_of_week")))\
    .cache()

print(sdf.schema)
    
    
grouped_schema = sdf.schema.add("failed_attempt_date", StringType())\
                            .add("failed_day_of_month", IntegerType())\
                            .add("failed_response_code", StringType())\
                            .add("failed_response_message", StringType())\
                            .add("failed_decline_type", StringType())\
                            .add("days_between", IntegerType())\
                            .add("failed_event_uuid", StringType())\
                            .add("failed_cc_expiration_date", StringType())\
                            .add("expiration_date_changed", BooleanType())\
                            .add("processor_mid_changed", StringType())\
                            .add("failed_cid", StringType())\
                            .add("failed_payment_service_id", StringType())\
                            .add("failed_merchant_number", StringType())\
                            .add("failed_bank_code", StringType())\
                            .add("bank_code_changed", BooleanType())\
                            .add("failed_bin", StringType())\
                            .add("bin_changed", BooleanType())
                            
                            
@pandas_udf(grouped_schema, PandasUDFType.GROUPED_MAP)
def merge_by_sub_spark(spark_df):
    pdf = pd.DataFrame(spark_df)
    result = pdf.groupby(['subscription_id', 'subsegment_id']).apply(merge_by_next_calendar_attempt)
    return result                            
                            
time_start = time.time()

def _map_to_pandas(rdds):
    """ Needs to be here due to pickling issues """
    return [pd.DataFrame(list(rdds))]

def toPandas(df, n_partitions=None):
    """
    Returns the contents of `df` as a local `pandas.DataFrame` in a speedy fashion. The DataFrame is
    repartitioned if `n_partitions` is passed.
    :param df:              pyspark.sql.DataFrame
    :param n_partitions:    int or None
    :return:                pandas.DataFrame
    """
    if n_partitions is not None: df = df.repartition(n_partitions)
    df_pand = df.rdd.mapPartitions(_map_to_pandas).collect()
    df_pand = pd.concat(df_pand)
    df_pand.columns = df.columns
    return df_pand

sdf_filtered = sdf\
        .filter(sdf.renew_att_num.isNotNull())\
        .groupby("subscription_id", "subsegment_id")\
        .apply(merge_by_sub_spark)\
        .filter("event_uuid != ''")  #.toPandas()

# pdf_all = toPandas(sdf_filtered, 128)  
pdf_all = sdf_filtered.toPandas()
print("# total time:", time.time() - time_start)      # total time: 2364.714904308319
    
pdf_all.shape  # previous: (6608320, 53)

StructType(List(StructField(received_date,DateType,true),StructField(site_id,StringType,true),StructField(subscription_id,StringType,true),StructField(transaction_id,StringType,true),StructField(transaction_date,TimestampType,true),StructField(added_expiry_years,StringType,true),StructField(bank_code,StringType,true),StructField(bank_name,StringType,true),StructField(billing_country,StringType,true),StructField(bin,StringType,true),StructField(card_brand,StringType,true),StructField(card_category,StringType,true),StructField(card_class,StringType,true),StructField(card_usage,StringType,true),StructField(cc_expiration_date,StringType,true),StructField(cid,StringType,true),StructField(date_increment,StringType,true),StructField(day_of_month,IntegerType,true),StructField(day_of_week,StringType,true),StructField(event_uuid,StringType,true),StructField(exp_setting_id,StringType,true),StructField(funding_source,StringType,true),StructField(issuer_country,StringType,true),StructField(merchant

(2383667, 59)

In [35]:
# import datetime as dt
# date = datetime.strptime('2018-11-10 10:55:31', '%Y-%m-%d %H:%M:%S')
# date =  datetime.datetime.strptime('2019-01-01 00:00:00', '%Y-%m-%d %H:%M:%S')
pdf_2019 = pdf_all[pdf_all['received_date'] >= datetime.date(year=2019,month=1,day=1)]
# pdf_2019.to_csv(WORK_DIR + 'calendar_retry_2019_01_to_03.csv')

In [16]:
pdf_all[pdf_all['transaction_date_in_string'].str.contains('2019-01', regex=False)]['received_date'].value_counts()

2019-01-12    27077
2019-01-04    26193
2019-01-17    25949
2019-01-25    25226
2019-01-18    24488
2019-01-05    24325
2019-01-11    23491
2019-01-03    23436
2019-01-10    23224
2019-01-02    23127
2019-01-20    23044
2019-01-29    22912
2019-01-31    22898
2019-01-27    22806
2019-01-09    22659
2019-01-22    22532
2019-01-16    22358
2019-01-26    22357
2019-01-24    22304
2019-01-15    22002
2019-01-19    21972
2019-01-06    21952
2019-01-23    21810
2019-01-28    21743
2019-01-08    21689
2019-01-30    21510
2019-01-01    21359
2019-01-13    21335
2019-01-21    20791
2019-01-14    20502
2019-01-07    19690
Name: received_date, dtype: int64

In [7]:
pdf_all.to_csv(WORK_DIR + 'calendar_retry_2019_01_to_02.csv')

2019-01-12    26998
2019-01-17    25938
2019-01-25    25223
2019-01-18    24482
2019-01-11    23404
2019-01-20    23039
2019-01-29    22907
2019-01-31    22898
2019-01-27    22797
2019-01-22    22528
2019-01-26    22355
2019-01-16    22352
2019-01-24    22302
2019-01-15    21999
2019-01-19    21968
2019-01-23    21807
2019-01-28    21733
2019-01-30    21509
2019-01-13    21306
2019-01-21    20784
2019-01-14    20473
2019-01-09    19618
2019-01-10    19347
2019-01-08    18449
2019-01-07    10761
2019-01-06    10736
2019-01-05     4550
2019-01-04     2668
2019-01-03     1043
2019-01-02       66
Name: received_date, dtype: int64

In [9]:
pdf_all.to_csv(WORK_DIR + 'calendar_retry_2019_01.csv')

In [19]:
pdf_all['transaction_date_in_string'].str.contains('2019-02', regex=False)

0          True
1          True
2          True
3          True
4          True
5         False
6          True
7          True
8          True
9         False
10         True
11         True
12         True
13        False
14         True
15         True
16         True
17         True
18        False
19         True
20         True
21        False
22        False
23         True
24         True
25         True
26         True
27         True
28         True
29         True
          ...  
737537     True
737538     True
737539     True
737540     True
737541     True
737542     True
737543     True
737544     True
737545     True
737546     True
737547     True
737548    False
737549    False
737550     True
737551    False
737552     True
737553    False
737554    False
737555     True
737556     True
737557     True
737558     True
737559     True
737560     True
737561    False
737562     True
737563     True
737564     True
737565     True
737566     True
Name: transaction_date_i

In [20]:
pdf_all[pdf_all['transaction_date_in_string'].str.contains('2019-01', regex=False)]['received_date'].value_counts()

2019-01-31    22899
2019-01-27    22431
2019-01-29    21548
2019-01-30    21507
2019-01-28    20827
2019-01-25    19297
2019-01-26    18547
2019-01-24    16043
2019-01-23    11855
2019-01-22     7977
2019-01-21     7248
2019-01-20     4183
2019-01-19     1732
2019-01-18      435
2019-01-17       47
Name: received_date, dtype: int64

In [24]:
pdf_all[pdf_all['transaction_date_in_string'].str.contains('2019-01', regex=False)]['received_date'].value_counts()

In [23]:
pdf_all['processor_mid_changed'].value_counts()

processor_and_mid_unchanged        597609
processor_changed_mid_changed      121453
processor_unchanged_mid_changed     18882
Name: processor_mid_changed, dtype: int64

In [7]:
# pdf_all.to_csv(WORK_DIR + 'calendar_retry_01_to_12.csv')
# pdf_all.to_csv(WORK_DIR + 'calendar_retry_2019_01_to_02.csv')
# pdf_all.to_csv(WORK_DIR + 'calendar_retry_2019_03.csv')

In [16]:
gp = pdf_all.groupby(['failed_decline_type'])
df_gp = gp.agg({'success':['sum', 'count']})
df_gp['success_rate'] = df_gp[('success', 'sum')] / df_gp[('success', 'count')] * 100

df_gp = df_gp.sort_values(by=['success_rate',('success', 'count')], ascending=False)
df_gp[df_gp[('success', 'count')] > 500]


success          success_rate
                               sum    count             
failed_decline_type                                     
transaction refused[002]       278      805    34.534161
attempt_lower_amount         10613    30847    34.405291
Insufficient Funds          316596  1094947    28.914276
acquirer error                 528     1920    27.500000
system error                   673     3267    20.599939
do_not_try_merchant_review      75      507    14.792899
server error                   178     1240    14.354839
invalid merchant               232     1875    12.373333
do_not_honor                113020   953804    11.849395
declined                     34922   302774    11.534015
txn_refused                 108174  1194203     9.058259
expired_card                 15328   173398     8.839779
no charge model found          397     4723     8.405674
new account info               272     3417     7.960199
Base                         59192  1125512     5.259118
Hard Fraud                   44783  1029955     4.348054
not_permitted                 1124    26103     4.306018
correct_cc_retry              8650   212810     4.064659
invalid_txn                   4829   126561     3.815551
Stop Recurring                 132     4272     3.089888
blocked_first_used             781    26387     2.959791
restricted_card                105     3625     2.896552
invalid_cc                    2555   127784     1.999468
invalid_account               2486   149289     1.665227
transaction refused[30]         50     4775     1.047120

In [24]:
gp = pdf_all.groupby(['expiration_date_changed'])
df_gp = gp.agg({'success':['sum', 'count']})
df_gp['success_rate'] = df_gp[('success', 'sum')] / df_gp[('success', 'count')] * 100

df_gp = df_gp.sort_values(by=['success_rate',('success', 'count')], ascending=False)
df_gp[df_gp[('success', 'count')] > 500]


success          success_rate
                            sum    count             
expiration_date_changed                              
True                     185706  1178591    15.756611
False                    540527  5427312     9.959387

In [43]:
pdf_all[pdf_all['bank_code'] == 'RB']['bin'].value_counts(dropna=False)

497040    1002
497355     409
513770     388
513162     383
497203     296
497049     284
497301     268
497490     266
497831     244
497874     233
456140     219
497043     172
497278     160
513283     153
497401     148
529097     144
513379     143
497886     142
497410     142
497202     123
497402     121
513286     120
456188     119
497783     117
497291     109
513781     103
513780     100
497827      99
497887      98
497041      93
          ... 
456279       1
513142       1
456101       1
513411       1
497551       1
456193       1
513552       1
513712       1
477181       1
497226       1
497655       1
453397       1
497683       1
497455       1
531442       1
459395       1
497271       1
513705       1
497576       1
497569       1
497732       1
449954       1
535029       1
497306       1
523718       1
513854       1
497908       1
513537       1
497705       1
420118       1
Name: bin, Length: 632, dtype: int64

In [46]:
pdf_all[pdf_all['bin'] == '497049']['bank_code'].value_counts(dropna=False)

NON3DS              5917
RB                   284
180603                10
180505                10
180509                 8
180531                 8
180613                 7
181205                 7
180520                 7
180517                 7
180607                 7
180610                 7
180513                 7
181124                 6
180516                 6
180819                 6
180530                 6
180523                 6
180911                 6
180713                 6
180907                 6
181201                 6
180620                 6
181212                 6
181128                 6
180606                 6
180525                 5
180512                 5
180831                 5
180526                 5
                    ... 
ED59D623D17C4406       1
180807                 1
180704                 1
D6EBDBFD6F511939       1
1A571B8A97C60472       1
9B4B5887AAE7DD89       1
180804                 1
558E0353D86A0463       1
180717                 1


In [44]:
gp = pdf_all.groupby(['bin', 'bank_code'])
df_gp = gp.agg({'success':['sum', 'count']})
df_gp['success_rate'] = df_gp[('success', 'sum')] / df_gp[('success', 'count')] * 100

df_gp = df_gp.sort_values(by=['success_rate',('success', 'count')], ascending=False)
df_gp[df_gp[('success', 'count')] > 500]


success        success_rate
                     sum  count             
bin    bank_code                            
497040 RB            836   1002    83.433134
447624 NON3DS        253    595    42.521008
535583 NON3DS        278    658    42.249240
437772 NON3DS        463   1117    41.450313
542011 NON3DS        428   1076    39.776952
428083 NON3DS        224    565    39.646018
553691 NON3DS       1772   4536    39.065256
479087 NON3DS        506   1308    38.685015
510070 NON3DS        492   1304    37.730061
554827 NON3DS        790   2127    37.141514
427649 NON3DS        240    657    36.529680
427427 NON3DS       1030   2820    36.524823
555949 NON3DS       1722   4765    36.138510
408397 NON3DS        321    891    36.026936
427654 NON3DS        348    983    35.401831
427638 NON3DS       8386  23829    35.192413
526911 NON3DS        263    751    35.019973
552175 NON3DS        285    818    34.841076
427670 NON3DS        254    730    34.794521
546952 NON3DS        234    673    34.769688
546938 NON3DS       3559  10255    34.705022
427655 NON3DS       2378   6868    34.624345
427938 NON3DS       1803   5213    34.586610
546955 NON3DS       1409   4093    34.424627
427644 NON3DS        446   1298    34.360555
489042 NON3DS        186    542    34.317343
511187 NON3DS        177    522    33.908046
449914 NON3DS       1208   3565    33.884993
514017 NON3DS        179    530    33.773585
536354 NON3DS        543   1609    33.747669
...                  ...    ...          ...
427685 NON3DS          0    863     0.000000
469198 NON3DS          0    844     0.000000
498792 NON3DS          0    831     0.000000
533136 NON3DS          0    810     0.000000
432090 NON3DS          0    790     0.000000
497566 NON3DS          0    765     0.000000
427686 NON3DS          0    748     0.000000
476361 NON3DS          0    746     0.000000
513606 NON3DS          0    741     0.000000
407513 NON3DS          0    724     0.000000
536610 NON3DS          0    706     0.000000
517537 NON3DS          0    696     0.000000
470350 NON3DS          0    691     0.000000
469197 NON3DS          0    687     0.000000
526415 NON3DS          0    679     0.000000
522358 NON3DS          0    671     0.000000
440847 NON3DS          0    670     0.000000
485446 NON3DS          0    658     0.000000
538404 NON3DS          0    619     0.000000
497784 NON3DS          0    609     0.000000
539614 NON3DS          0    587     0.000000
529615 NON3DS          0    576     0.000000
436393 NON3DS          0    573     0.000000
518810 NON3DS          0    571     0.000000
377171 NON3DS          0    569     0.000000
529926 NON3DS          0    566     0.000000
492505 NON3DS          0    564     0.000000
497237 NON3DS          0    509     0.000000
451084 NON3DS          0    504     0.000000
513368 NON3DS          0    501     0.000000

[1427 rows x 3 columns]

In [50]:
gp = pdf_all[pdf_all['bank_code'] == 'NON3DS'].groupby(['day_of_month'])
df_gp = gp.agg({'success':['sum', 'count']})
df_gp['success_rate'] = df_gp[('success', 'sum')] / df_gp[('success', 'count')] * 100

df_gp = df_gp.sort_values(by=['success_rate',('success', 'count')], ascending=False)
df_gp[df_gp[('success', 'count')] > 100]


success         success_rate
                 sum   count             
day_of_month                             
4              21556  124566    17.304883
5              17053  114910    14.840310
11             18065  122743    14.717744
3              15552  108650    14.313852
2              15550  109193    14.240840
1              15494  110037    14.080718
10             16171  115357    14.018222
12             17104  122788    13.929700
30             15483  113592    13.630361
28             16663  122492    13.603337
9              15773  116091    13.586755
16             15938  117555    13.557909
8              14693  108460    13.546930
18             16201  119634    13.542137
19             15606  116689    13.374011
29             14957  112323    13.316062
31              9254   69898    13.239292
13             14294  108303    13.198157
15             14729  112052    13.144790
27             15715  120619    13.028627
17             15841  121821    13.003505
14             14298  109955    13.003501
26             15695  120776    12.995131
20             14421  111726    12.907470
22             14994  116498    12.870607
25             15775  122580    12.869147
21             14608  114389    12.770459
24             14926  117762    12.674717
23             14805  118486    12.495147
7              13094  105517    12.409375
6              12465  103659    12.025005

In [23]:
gp = pdf_all.groupby(['processor_mid_changed'])
df_gp = gp.agg({'success':['sum', 'count']})
df_gp['success_rate'] = df_gp[('success', 'sum')] / df_gp[('success', 'count')] * 100

df_gp = df_gp.sort_values(by=['success_rate',('success', 'count')], ascending=False)
df_gp[df_gp[('success', 'count')] > 500]


success          success_rate
                                    sum    count             
processor_mid_changed                                        
processor_changed_mid_changed    476988  1082366    44.069012
processor_and_mid_unchanged      245475  5428336     4.522104
processor_unchanged_mid_changed    3731    93393     3.994946
processor_changed_mid_unchanged      39     1808     2.157080

In [35]:
# len(pdf_all['failed_response_message'].unique()) #1149
# len(pdf_all['merchant_number'].unique())
# len(pdf_all['merchant_number'].unique())  #181

for field in pdf_all.columns:
    print("{}: {} ".format(field, len(pdf_all[field].unique())))

received_date: 365 
site_id: 90 
subscription_id: 2644026 
transaction_id: 6605903 
transaction_date: 2418551 
added_expiry_years: 21 
bank_code: 29538 
bank_name: 8281 
billing_country: 244 
bin: 43240 
card_brand: 7 
card_category: 107 
card_class: 7 
card_usage: 24 
cc_expiration_date: 497 
cid: 8 
date_increment: 12 
day_of_month: 31 
day_of_week: 7 
event_uuid: 6605903 
exp_setting_id: 56548 
funding_source: 8 
issuer_country: 196 
merchant_descriptor: 51 
merchant_number: 181 
mid_entity_code: 7 
new_status: 6 
order_entity_code: 7 
payment_amount: 15330 
payment_amount_usd: 194529 
payment_currency: 43 
payment_method_id: 9 
payment_service_id: 18 
renew_att_num: 16 
request_amount: 15320 
request_amount_usd: 194520 
request_currency: 43 
requisition_id: 6605903 
response_code: 96 
response_message: 383275 
status: 5 
subsegment_id: 2912334 
transaction_date_in_string: 2418489 
transaction_hour: 28 
transaction_type: 1 
success: 2 
failed_attempt_date: 2625330 
failed_day_of_mon

In [37]:
response_message_list =  pdf_all['response_message'].unique().tolist()
failed_response_message_list =  pdf_all['failed_response_message'].unique().tolist()

set(response_message_list).symmetric_difference(set(failed_response_message_list))

{'SUCCESS[00] [227925] [] []',
 'SUCCESS[00] [406855] [] []',
 'SUCCESS[00] [049434] [124800] [124800]',
 'SUCCESS[00] [000119] [] []',
 'SUCCESS[00] [006719] [] []',
 'SUCCESS[00] [064416] [] []',
 'SUCCESS[00] [346843] [] []',
 'SUCCESS[00] [110948] [] []',
 'SUCCESS[00] [60R798] [] []',
 'SUCCESS[00] [T44090] [] []',
 'SUCCESS[00] [236603] [] []',
 'SUCCESS[00] [081896] [111800] [111800]',
 'SUCCESS[00] [256095] [] []',
 'SUCCESS[00] [278492] [] []',
 'SUCCESS[00] [380411] [] []',
 'SUCCESS[00] [000794] [111800] [111800]',
 'SUCCESS[00] [399859] [] []',
 'SUCCESS[00] [277393] [] []',
 'SUCCESS[00] [247599] [] []',
 'SUCCESS[00] [003386] [] []',
 'SUCCESS[00] [285343] [111800] [111800]',
 'SUCCESS[00] [644128] [] []',
 'SUCCESS[00] [402128] [] []',
 'SUCCESS[00] [300760] [] []',
 'SUCCESS[00] [529393] [111800] [111800]',
 'SUCCESS[00] [T06830] [] []',
 'SUCCESS[00] [762310] [111800] [111800]',
 'SUCCESS[00] [524671] [] []',
 'SUCCESS[00] [019671] [] []',
 'SUCCESS[00] [398130] [11180

In [18]:
gp = pdf_all[pdf_all['failed_decline_type'] == 'do_not_honor'].groupby(['failed_response_message'])
df_gp = gp.agg({'success':['sum', 'count']})
df_gp['success_rate'] = df_gp[('success', 'sum')] / df_gp[('success', 'count')] * 100

df_gp = df_gp.sort_values(by=['success_rate',('success', 'count')], ascending=False)
df_gp[df_gp[('success', 'count')] > 500]

success         success_rate
                            sum   count             
failed_response_message                             
7: Do Not Honor           79029  402478    19.635607
349: Do Not Honor         24839  397089     6.255273
005: Do not honor          9132  153548     5.947326
05: Do Not Honor             20     685     2.919708

In [10]:
pdf_all['expiration_date_changed'].value_counts()

False    5427312
True     1178591
Name: expiration_date_changed, dtype: int64

In [14]:
gp = pdf_all.groupby(['expiration_date_changed'])
df_gp = gp.agg({'success':['sum', 'count']})
df_gp['success_rate'] = df_gp[('success', 'sum')] / df_gp[('success', 'count')] * 100

df_gp = df_gp.sort_values(by=['success_rate',('success', 'count')], ascending=False)
df_gp.columns = df_gp.columns.droplevel(0)
df_gp[df_gp['count'] > 1].rename(columns={'sum': 'success', 'count': 'overall', '':'success_rate'})

,success,overall,success_rate
expiration_date_changed,,,
True,185706,1178591,15.756611
False,540527,5427312,9.959387


In [13]:
gp = pdf_all.groupby(['processor_mid_changed'])
df_gp = gp.agg({'success':['sum', 'count']})
df_gp['success_rate'] = df_gp[('success', 'sum')] / df_gp[('success', 'count')] * 100

df_gp = df_gp.sort_values(by=['success_rate',('success', 'count')], ascending=False)
df_gp.columns = df_gp.columns.droplevel(0)
df_gp[df_gp['count'] > 1].rename(columns={'sum': 'success', 'count': 'overall', '':'success_rate'})

,success,overall,success_rate
processor_mid_changed,,,
processor_changed_mid_changed,476988,1082366,44.069012
processor_and_mid_unchanged,245475,5428336,4.522104
processor_unchanged_mid_changed,3731,93393,3.994946
processor_changed_mid_unchanged,39,1808,2.157080


In [12]:
gp = pdf_all.groupby(['failed_decline_type'])
df_gp = gp.agg({'success':['sum', 'count']})
df_gp['success_rate'] = df_gp[('success', 'sum')] / df_gp[('success', 'count')] * 100

df_gp = df_gp.sort_values(by=['success_rate',('success', 'count')], ascending=False)
df_gp.columns = df_gp.columns.droplevel(0)
df_gp[df_gp['count'] > 1].rename(columns={'sum': 'success', 'count': 'overall', '':'success_rate'})

,success,overall,success_rate
failed_decline_type,,,
no reply,10,11,90.909091
issuer unavailable,141,337,41.839763
international filtering,2,5,40.000000
corrupt input data,4,11,36.363636
attempt_lower_amount,10613,30847,34.405291
unable to determine format,52,174,29.885057
Insufficient Funds,316596,1094947,28.914276
acquirer error,528,1920,27.500000
ioexception,7,33,21.212121


In [70]:
pdf_all['days_between'].value_counts(dropna=False)

7      2314714
5      1886071
10      938895
3       481281
4       469445
2       260902
8       136494
6        59791
1        33262
14       11420
9         3680
11        3226
30        2648
0         1942
15        1270
12         483
13         235
31          45
27          18
17           8
25           8
29           6
28           6
19           5
21           5
18           4
20           3
16           3
22           3
360          2
26           2
91           2
101          1
102          1
119          1
183          1
168          1
184          1
185          1
223          1
300          1
301          1
92           1
57           1
88           1
87           1
85           1
54           1
23           1
24           1
61           1
60           1
37           1
38           1
42           1
362          1
Name: days_between, dtype: int64

In [29]:
from IPython.display import display
pd.options.display.max_columns = None
pdf_all_05.head()

,received_date,site_id,subscription_id,transaction_id,transaction_date,added_expiry_years,bank_code,bank_name,billing_country,bin,card_brand,card_category,card_class,card_usage,cc_expiration_date,cid,date_increment,day_of_month,day_of_week,event_uuid,exp_setting_id,funding_source,issuer_country,merchant_descriptor,merchant_number,mid_entity_code,new_status,order_entity_code,payment_amount,payment_amount_usd,payment_currency,payment_method_id,payment_service_id,renew_att_num,request_amount,request_amount_usd,request_currency,requisition_id,response_code,response_message,status,subsegment_id,transaction_date_in_string,transaction_hour,transaction_type,success,failed_attempt_date,failed_day_of_month,failed_response_code,failed_response_message,failed_decline_type,days_between,failed_event_uuid
0,2018-05-10,adbehru,10002261401,6761535723,2018-05-10 00:59:03,,NON3DS,SBERBANK of Russia,RU,427609,Visa,Visa Classic,Consumer,NON USA Consumer Debit - No Pin Access,1018,RETRY_DECLINED.2,,10,thursday,78606856-da8a-4752-b64b-4455475f4cff,,Debit,RU,DRI*Adobe Systems,1411163460,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,759.92,12.049068,RUB,Visa,netgiro-bms,4,759.92,12.049068,RUB,47764749400,27054,Attempt lower amount.[51] [] [111800] [111800],New,15753794300,2018-05-10 00:59:03,00,Authorize,0,2018-05-03 00:59:51,3,27054,Attempt lower amount.[51] [] [111800] [111800],Insufficient Funds,7,d8761448-9c42-423e-83a3-40f470801400
1,2018-05-17,adbehru,10002261401,6788536463,2018-05-17 00:57:42,,NON3DS,SBERBANK of Russia,RU,427609,Visa,Visa Classic,Consumer,NON USA Consumer Debit - No Pin Access,1018,RETRY_DECLINED.2,,17,thursday,981ea2b5-8ad9-42d8-aab5-1bd5c16c7bb7,,Debit,RU,DRI*Adobe Systems,1411163460,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,759.92,12.16736,RUB,Visa,netgiro-bms,5,759.92,12.16736,RUB,47992656800,27054,Attempt lower amount.[51] [] [111800] [111800],New,15753794300,2018-05-17 00:57:42,00,Authorize,0,2018-05-10 00:59:03,10,27054,Attempt lower amount.[51] [] [111800] [111800],Insufficient Funds,7,78606856-da8a-4752-b64b-4455475f4cff
2,2018-05-19,adbehru,10002261401,6796959663,2018-05-19 00:55:38,,NON3DS,SBERBANK of Russia,RU,427609,Visa,Visa Classic,Consumer,NON USA Consumer Debit - No Pin Access,1018,RETRY_DECLINED.2,,19,saturday,0d1fbb95-019b-4f5f-8c53-f96ae78c2d0a,,Debit,RU,DRI*Adobe Systems,1411163460-RUB-pacific,DR_IRELAND-ENTITY,Failed,DR_IRELAND-ENTITY,759.92,12.227846,RUB,Visa,netgiro-bms,6,759.92,12.227846,RUB,48060561100,27083,27083: Insufficient Funds[51] [] [111800] [111...,Declined,15753794300,2018-05-19 00:55:38,00,Authorize,0,2018-05-17 00:57:42,17,27054,Attempt lower amount.[51] [] [111800] [111800],Insufficient Funds,2,981ea2b5-8ad9-42d8-aab5-1bd5c16c7bb7
3,2018-05-26,adbehap,10004467401,6823936303,2018-05-26 01:02:48,,NON3DS,Prisma Medios de Pago S.A.,AR,402918,Visa,Visa Classic,Consumer,True credit (No PIN/Signature capability),0219,,,26,saturday,de96f214-5f4a-4556-bffa-efbf399a3cc2,,Credit,AR,DRI*Adobe Sales,1156962062,DR_IRELAND-ENTITY,Completed,DR_IRELAND-ENTITY,88.83,4.14944,ARS,Visa,netgiro-seb,2,88.83,4.14944,ARS,48293140500,0,SUCCESS[00] [648888] [] [],New,15973691300,2018-05-26 01:02:48,01,Authorize,1,2018-05-19 00:55:15,19,27083,27083: 27083: Insufficient Funds[51] [] [11180...,Insufficient Funds,7,a7fa940f-10b4-4e5e-97de-efb9d07dbc20
4,2018-05-09,tmapac,10013587701,6760540633,2018-05-09 13:00:03,,NON3DS,ANZ Bank New Zealand Limited,NZ,436773,Visa,Visa Classic,Consumer,True credit (No PIN/Signature capability),0820,,,9,wednesday,f56e6e63-b571-40a4-8814-0bb1562d8c23,,Credit,NZ,DRI*Trend Micro,1411163460,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,39.95,27.863359,NZD,Visa,netgiro-bms,2,39.95,27.863359,NZD,47760042300,27050,Do not try again/use alternate payment card.[0...,New,13289071100,2018-05-09 13:00:03,13,Authorize,0,2018-05-04 00:57:27,4,27050,Do not try again/use alternate payment card.[0...,Hard Fraud,5,19f1ba04-ffa3-4e0c-a717-af037da6ab9d


In [31]:
import src.web.utils as ut

def is_day_of_month_correct(row):
#     txn_date_day = ut.to_day(row['transaction_date_in_string'])
#     return row['day_of_month'] == txn_date_day
    txn_date_day = ut.to_day(row['failed_attempt_date'])
    return row['failed_day_of_month'] == txn_date_day

# ut.to_day('2018-05-03 02:05:51')
# pdf_all[pdf_all['subscription_id'] == '11456523001'].apply(is_day_of_month_correct, axis=1)

In [32]:
pdf_all['is_day_of_month_correct'] = pdf_all.apply(is_day_of_month_correct, axis=1)

In [35]:
pdf_all[ (pdf_all['is_day_of_month_correct'] == False) & (pdf_all['failed_attempt_date'] < '2018-05-01')].head(5)

,received_date,site_id,subscription_id,transaction_id,transaction_date,added_expiry_years,bank_code,bank_name,billing_country,bin,card_brand,card_category,card_class,card_usage,cc_expiration_date,cid,date_increment,day_of_month,day_of_week,event_uuid,exp_setting_id,funding_source,issuer_country,merchant_descriptor,merchant_number,mid_entity_code,new_status,order_entity_code,payment_amount,payment_amount_usd,payment_currency,payment_method_id,payment_service_id,renew_att_num,request_amount,request_amount_usd,request_currency,requisition_id,response_code,response_message,status,subsegment_id,transaction_date_in_string,transaction_hour,transaction_type,success,failed_attempt_date,failed_day_of_month,failed_response_code,failed_response_message,failed_decline_type,days_between,failed_event_uuid,Mean,Min,Max,StdDev,Max_95,Max_99,Min_1,is_day_of_month_correct
24953,2018-01-07,adbehbr,3872264201,6350634843,2018-01-07 06:53:16,,3FE107B8E28439A3,ITAU UNIBANCO S.A.,BR,520977,MasterCard,(Unembossed) Standard MasterCard Card,Consumer,Credit Hybrid (meaning it has pin capability a...,0218,,,7,sunday,e8442337-487a-4eac-96c2-72fa03fed96d,,Credit,BR,DRI*Adobe Systems,75FA81DE-5AC5-5DCF-E97C-8F80A08ADE1E,DR_BRAZIL-ENTITY,Declined,DR_BRAZIL-ENTITY,35,10.79384,BRL,MasterCard,pagador,2,35,10.79384,BRL,44125468100,530,7: Do Not Honor,New,14761975300,2018-01-07 00:53:16,00,Authorize,0,2018-01-02 01:19:45,1,530,7: Do Not Honor,do_not_honor,5,ebf80761-c9d1-41e8-a717-bfb5f8007a18,19.677476,2.49,101.585362,12.442595,46.65000,54.154381,9.000000,False
24954,2018-01-14,adbehbr,3872264201,6370866863,2018-01-14 06:39:53,,F7005EEDC0D7D358,ITAU UNIBANCO S.A.,BR,520977,MasterCard,(Unembossed) Standard MasterCard Card,Consumer,Credit Hybrid (meaning it has pin capability a...,1225,,,14,sunday,1b5acdae-9b8c-4bb8-920d-a62920d944e2,,Credit,BR,DRI*Adobe Systems,75FA81DE-5AC5-5DCF-E97C-8F80A08ADE1E,DR_BRAZIL-ENTITY,Completed,DR_BRAZIL-ENTITY,35,10.903642,BRL,MasterCard,pagador,3,35,10.903642,BRL,44303869400,100,0:Successful,New,14761975300,2018-01-14 00:39:53,00,Authorize,1,2018-01-07 00:53:16,1,530,7: Do Not Honor,do_not_honor,7,e8442337-487a-4eac-96c2-72fa03fed96d,19.677476,2.49,101.585362,12.442595,46.65000,54.154381,9.000000,False
27405,2018-01-07,adbehkr,8102301101,6350559023,2018-01-07 06:46:31,,NON3DS,"KB Kookmin Card Co., Ltd.",KR,467309,Visa,Visa Classic,Consumer,NON USA Consumer Debit - No Pin Access,1120,,,7,sunday,9bf031f5-e8fd-4051-a597-10019eb73555,,Debit,KR,DRI*Adobe Systems,1156962062,DR_IRELAND-ENTITY,Completed,DR_KOREA-ENTITY,11000,10.341477,KRW,Visa,netgiro-seb,2,11000,10.341477,KRW,44124541500,0,SUCCESS[00] [306722] [] [],New,14802316100,2018-01-07 00:46:31,00,Authorize,1,2018-01-02 01:05:16,1,27054,Attempt lower amount.[51] [] [111800] [111800],Insufficient Funds,5,e66d96d3-1d62-420b-a1c1-9c1f0d1f2580,20.165271,2.60,2881.529855,23.446634,52.70000,55.699431,9.797404,False
87261,2018-01-12,techsm,2598158009,6364834953,2018-01-12 06:36:27,,4A41F258B5903040,None,US,558174,None,None,None,None,0119,,,12,friday,1d466a9a-11a9-4a9d-aec1-7455bc34f531,,None,US,DRI*TechSmith,941000108061_00000004,C5_INC-ENTITY,Completed,DR_INC-ENTITY,9.95,9.95,USD,MasterCard,mes,2,9.95,9.95,USD,44250640400,100,SUCCESS No Match,New,14800380100,2018-01-12 00:36:27,00,Authorize,1,2018-01-07 00:29:03,1,530,349: Do Not Honor,do_not_honor,5,0fc241bf-fd2b-4f36-92e0-3194ffe5de70,51.862588,2.49,1575.000000,64.466072,147.38500,190.000000,7.470000,False
87861,2018-01-28,adbehap,2924551709,6411091163,2018-01-28 06:37:09,,NON3DS,PT Bank Mandiri (Persero) TBK,ID,413719,Visa,Visa Gold,Consumer,True credit (No PIN/Signature capability),1021,,,28,sunday,563785f9-561b-4b12-a40b-a83f5128d601,,Credit,ID,DRI*Adobe Sales,1156962062,DR_IRELAND-ENTITY,Completed,DR_IRELAND-ENTITY,134800,10.123923,IDR,Visa,netgiro-seb,2,134800,10.123923,IDR,44663775400,0,SUCCESS[00] [137365] [] [],New,14960424200,2018-01-28 00:37:09,00,Authorize,1,2018-01-23 00:33:57,1,27001,Transaction refused[51] [000000] [] [],In

In [12]:
pdf_all['is_day_of_month_correct'].value_counts(normalize=True)

True     0.903584
False    0.096416
Name: is_day_of_month_correct, dtype: float64

In [55]:
pdf_all[pdf_all['subscription_id'] == '11456523001'][['received_date', 'subscription_id', 'day_of_month', 'transaction_date', 'transaction_date_in_string']]

,received_date,subscription_id,day_of_month,transaction_date,transaction_date_in_string
11038,2018-04-18,11456523001,18,2018-04-18 04:04:30,2018-04-18 04:04:30
11039,2018-04-24,11456523001,24,2018-04-24 04:00:11,2018-04-24 04:00:11
11040,2018-04-28,11456523001,28,2018-04-28 04:09:09,2018-04-28 04:09:09
11041,2018-05-01,11456523001,5,2018-05-01 15:56:13,2018-05-01 15:56:13
11042,2018-05-02,11456523001,5,2018-05-02 02:06:51,2018-05-02 02:06:51
11043,2018-05-03,11456523001,5,2018-05-03 02:05:51,2018-05-03 02:05:51
11044,2018-05-03,11456523001,5,2018-05-03 09:26:57,2018-05-03 09:26:57


In [71]:
pdf_all[(pdf_all['days_between'] == 0) & (pdf_all['transaction_date'] >= '2018-07-01') & (pdf_all['transaction_date'] <= '2018-07-31')]

,received_date,site_id,subscription_id,transaction_id,transaction_date,added_expiry_years,bank_code,bank_name,billing_country,bin,card_brand,card_category,card_class,card_usage,cc_expiration_date,cid,date_increment,day_of_month,day_of_week,event_uuid,exp_setting_id,funding_source,issuer_country,merchant_descriptor,merchant_number,mid_entity_code,new_status,order_entity_code,payment_amount,payment_amount_usd,payment_currency,payment_method_id,payment_service_id,renew_att_num,request_amount,request_amount_usd,request_currency,requisition_id,response_code,response_message,status,subsegment_id,transaction_date_in_string,transaction_hour,transaction_type,success,failed_attempt_date,failed_day_of_month,failed_response_code,failed_response_message,failed_decline_type,days_between,failed_event_uuid,is_day_of_month_correct
13134,2018-07-08,avast,11675960301,7025595723,2018-07-08 16:58:08,,NON3DS,The National Commercial Bank,SA,430260,Visa,Visa Platinum,Consumer,True credit (No PIN/Signature capability),0819,,,8,sunday,8eef6339-8523-4b3e-845b-fc08dcfe3eba,,Credit,SA,DRI*AVAST Software,1156962062,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,279,74.400121,SAR,Visa,netgiro-seb,2,279,74.400121,SAR,49744932400,27001,Transaction refused[51] [000000] [] [],New,16154325700,2018-07-08 11:58:08,11,Authorize,0,2018-07-08 02:29:31,8,27083,Insufficient Funds[51] [] [111800] [111800],Insufficient Funds,0,f9adb438-6556-43ca-a8af-3dacbc0b8005,True
13178,2018-07-08,avast,11679578401,7025606553,2018-07-08 17:03:42,,NON3DS,POSTE ITALIANE S.P.A. - Patrimonio Bancoposta,IT,402360,Visa,Visa Electron,Consumer,NON USA Consumer Debit - No Pin Access,0319,,,8,sunday,fe17ef4f-0dfe-43ae-85e8-7cc0192ca8f4,,Prepaid,IT,DRI*AVAST Software,1411163460,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,79.99,93.780276,EUR,Visa,netgiro-bms,2,79.99,93.780276,EUR,49744999500,27083,Insufficient Funds[51] [] [111800] [111800],New,16162839100,2018-07-08 12:03:42,12,Authorize,0,2018-07-08 03:16:56,8,27083,Insufficient Funds[51] [] [111800] [111800],Insufficient Funds,0,2a081ab5-3666-489d-80eb-90b2f6fc69a7,True
47622,2018-07-08,avast,11818930901,7025601533,2018-07-08 17:00:43,,NON3DS,GUARANTY TRUST BANK,NG,539983,MasterCard,Standard Debit MasterCard,Consumer,NON USA Consumer Debit - Pin Capable,0321,,,8,sunday,2b993208-9347-4e04-bb30-1b70a678e251,,Debit,NG,DRI*AVAST Software,1411163460,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,49.99,49.99,USD,MasterCard,netgiro-bms,2,49.99,49.99,USD,49744970000,27083,Insufficient Funds[51] [] [111800] [111800],New,16432503900,2018-07-08 12:00:43,12,Authorize,0,2018-07-08 02:47:13,8,27083,Insufficient Funds[51] [] [111800] [111800],Insufficient Funds,0,fa8d521e-f313-4047-9ac1-32d1df43f243,True
79246,2018-07-02,avast,11647408001,6998581633,2018-07-02 18:32:11,,NON3DS,Kiwibank Ltd,NZ,483741,Visa,Visa Classic,Consumer,NON USA Consumer Debit - No Pin Access,0720,,,2,monday,1ebf830f-1a51-4d43-b7b8-730b40fe4944,,Debit,NZ,DRI*AVAST Software,1411163460,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,86.24,58.29338,NZD,Visa,netgiro-bms,2,86.24,58.29338,NZD,49557443700,27083,Insufficient Funds[51] [] [111800] [111800],New,16102925200,2018-07-02 13:32:11,13,Authorize,0,2018-07-02 03:17:30,2,27083,Insufficient Funds[51] [] [111800] [111800],Insufficient Funds,0,18fa7ab9-55fd-47de-8734-1084e73ea342,True
139422,2018-07-08,ciscoctg,10871423201,7025586253,2018-07-08 16:46:47,,F9EA1BCBF1F961C7,,US,376945,American Express,,,True credit (No PIN/Signature capability),1022,,,8,sunday,24b61aba-c18a-4afb-9c8b-1cb330e7497f,,,,DRI*Cisco Systems,941000108063_00000001,DR_INC-ENTITY,Declined,C5_INC-ENTITY,41.73,41.73,USD,AmericanExpress,mes,3,41.73,41.73,USD,49744456000,530,005: Do not honor,New,16401428400,2018-07-08 11:46:47,11,Authorize,0,2018-07-08 11:42:38,8,530,51: DECLINED,declined,0,81dfab3c-9612-474c-a186-d749f9579bb8,True
145416,2018-07-02,avast,11644641501,6998580983,2018-07-02 18:31:35,,NON3DS,TURKIYE GARANTI BANKASI A.S.,TR,517041,MasterCard,Standard Debit MasterCard

In [80]:
pdf_all_07[pdf_all_07['subscription_id'] == 11675960301].sort_values(by=['transaction_date'])[['transaction_date', 'cid','transaction_date_in_string','renew_att_num', 'event_uuid']]

,transaction_date,cid,transaction_date_in_string,renew_att_num,event_uuid
1785021,2018-07-08 07:29:29,NaN,2018-07-08 02:29:29,1.0,69cc67d1-0e26-469e-9385-94fd6fbbebe2
92457,2018-07-08 07:29:31,RETRY_DECLINED.2,2018-07-08 02:29:31,1.0,f9adb438-6556-43ca-a8af-3dacbc0b8005
1230956,2018-07-08 16:58:08,NaN,2018-07-08 11:58:08,2.0,8eef6339-8523-4b3e-845b-fc08dcfe3eba
646111,2018-07-08 16:58:10,RETRY_DECLINED.2,2018-07-08 11:58:10,2.0,e12953ca-2e57-4bbe-bb72-bf56033cf3c6


In [70]:
# pdf_all[(pdf_all['days_between'] == 0) & (pdf_all['transaction_date'] >= '2018-07-01') & (pdf_all['transaction_date'] <= '2018-07-31')]
pdf_all['success'].value_counts(normalize=True)

0    0.892404
1    0.107596
Name: success, dtype: float64

In [8]:
pdf_all['days_between'].value_counts()

7      2316379
5      1886265
10      939059
3       481290
4       469473
2       261138
8       136517
6        59801
1        33269
14       11426
9         3689
11        3234
30        2649
0         1940
15        1270
12         483
13         242
31          45
27          18
17          10
25           8
21           7
28           7
29           6
19           5
173          4
18           4
16           4
20           3
22           3
        ...   
295          1
300          1
301          1
24           1
164          1
119          1
65           1
101          1
100          1
96           1
92           1
125          1
132          1
136          1
85           1
76           1
69           1
61           1
162          1
60           1
57           1
54           1
42           1
38           1
37           1
33           1
139          1
147          1
154          1
362          1
Name: days_between, Length: 90, dtype: int64

In [37]:
sdf_filtered

DataFrame[received_date: date, site_id: string, subscription_id: string, transaction_id: string, transaction_date: timestamp, added_expiry_years: string, bank_code: string, bank_name: string, billing_country: string, bin: string, card_brand: string, card_category: string, card_class: string, card_usage: string, cc_expiration_date: string, cid: string, date_increment: string, day_of_month: int, day_of_week: string, event_uuid: string, exp_setting_id: string, funding_source: string, issuer_country: string, merchant_descriptor: string, merchant_number: string, mid_entity_code: string, new_status: string, order_entity_code: string, payment_amount: string, payment_amount_usd: string, payment_currency: string, payment_method_id: string, payment_service_id: string, renew_att_num: string, request_amount: string, request_amount_usd: string, request_currency: string, requisition_id: string, response_code: string, response_message: string, status: string, subsegment_id: string, transaction_date_i

In [91]:
import itertools

# Merge failed attempt with the next attempt
def merge_by_sub(group):

    
    #Assign default values to additonal columns
    group['failed_attempt_date'] = ''
    group['failed_day_of_month'] = 0
    group['failed_response_code'] = ''
    group['failed_response_message'] = ''
    group['failed_decline_type'] = ''
    group['days_between'] = 0
    group['failed_event_uuid'] = ''    
    new_group = pd.DataFrame(columns = group.columns)
    
    sorted_group = group.sort_values(by=['transaction_date_in_string', 'cid'])
    initial_att_num = sorted_group.head(1)['renew_att_num'].iloc[0]
    last_att_num = sorted_group.tail(1)['renew_att_num'].iloc[0]
    if not last_att_num or not initial_att_num:
        group['event_uuid'] = '' 
        return group.iloc[0]
        
    for att_num in range(int(initial_att_num), int(last_att_num)):
        next_att_num = int(att_num) + 1
        initial_att_indexes = group[group['renew_att_num'] == str(att_num)].index.tolist()
        next_att_indexes = group[group['renew_att_num'] == str(next_att_num)].index.tolist()
        
        for item in itertools.product(*[initial_att_indexes, next_att_indexes]):
            merged = group.loc[item[1]].copy(deep=True)
            inital_row = group.loc[item[0]]                   
            failed_attempt_date = inital_row['transaction_date_in_string']                   
            merged['failed_attempt_date'] = failed_attempt_date

            merged['failed_day_of_month'] = inital_row['day_of_month']
            merged['day_of_month'] = int(merged['day_of_month'])
            merged['failed_response_code'] = inital_row['response_code']
            
            failed_response_msg = inital_row['response_message']
            merged['failed_response_message'] = failed_response_msg  

            merged['failed_decline_type'] = decline_type(inital_row['response_message']) 
            merged['days_between'] = days_between_ds(merged)
            merged['failed_event_uuid'] =   inital_row['event_uuid']   
            new_group = new_group.append(merged)
            
           
    if new_group.empty:
        group['event_uuid'] = '' 
        return group.iloc[0]
    
    return new_group

In [25]:
# Merge only the last two renew_att_num with the last single attempt
def merge_by_sub(group):

    
    #Assign default values to additonal columns
    group['failed_attempt_date'] = ''
    group['failed_day_of_month'] = 0
    group['failed_response_code'] = ''
    group['failed_response_message'] = ''
    group['failed_decline_type'] = ''
    group['days_between'] = 0
    group['failed_event_uuid'] = ''    
    
    last = pd.DataFrame() 
    df_success = group[group['success'] == 1]

    if not df_success.empty:
        df_last_attempt = df_success.iloc[0]
    else:
        sorted_group = group.sort_values(by=['transaction_date_in_string', 'cid'])
        df_last_fail = sorted_group.tail(1)
        if not df_last_fail.empty:
            df_last_attempt =  df_last_fail.iloc[0]
        else:
            group['event_uuid'] = '' 
            return group
    
    last_att_num = int(df_last_attempt['renew_att_num'])
    previous_att_num = last_att_num - 1
   
    df_failed_txn = group[(group['renew_att_num'].isin([ str(previous_att_num),  df_last_attempt['renew_att_num'] ])) & (group['event_uuid'] != df_last_attempt['event_uuid']) ]
    if df_failed_txn.empty:
        group['event_uuid'] = '' 
        return group
    
    last = last.append( [df_last_attempt] * len(df_failed_txn),ignore_index=True ).set_index(df_failed_txn.index)
        
    failed_attempt_date = df_failed_txn['transaction_date_in_string']
    
    last['failed_attempt_date'] = failed_attempt_date
    last['failed_day_of_month'] = df_failed_txn['day_of_month']
    last['day_of_month'] = last['day_of_month'].astype(int)
    last['failed_response_code'] = df_failed_txn['response_code']
    
    failed_response_msg = df_failed_txn['response_message']
    last['failed_response_message'] = failed_response_msg  
    
    last['failed_decline_type'] = df_failed_txn['response_message'].apply(decline_type) 
    last['days_between'] = last.apply(days_between_ds, axis=1)    
    last['failed_event_uuid'] =   df_failed_txn['event_uuid']   
        
    return last

In [6]:
# Merge failed attempt with the next attempt
import time

import pandas as pd


from pyspark.sql.types import *
from pyspark.sql.functions import col, count, rand, collect_list, explode, struct, count, lit, lower
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.functions import udf
from pyspark.sql.types import *
import pyspark_cassandra
        
import calendar, datetime
from datetime import timedelta
import itertools

WORK_DIR = '/var/spark/ml_files/'


def convert_status(value):
    if value == 'Completed':
        return 1
    else:
        return 0
    
    
def cast_to_int(value, default=-1):
    try:
        return int(value)
    except (ValueError, TypeError):
        return default


#Convert from str to datetime
def to_date(datestr):
    struct = time.strptime(datestr, "%Y-%m-%d %H:%M:%S")
    date = datetime.date(struct.tm_year,struct.tm_mon,struct.tm_mday)
    return date

#Get difference between d2 and d1 in days.
def days_between(d1, d2):
    d1 = to_date(d1)
    d2 = to_date(d2)
    return abs((d2 - d1).days)

#Determines the week (number) of the month
def week_of_month(datestr):
    date = to_date(datestr)
    #Calendar object. 6 = Start on Sunday, 0 = Start on Monday
    cal_object = calendar.Calendar(6)
    month_calendar_dates = cal_object.itermonthdates(date.year,date.month)

    day_of_week = 1
    week_number = 1

    for day in month_calendar_dates:
        #add a week and reset day of week
        if day_of_week > 7:
            week_number += 1
            day_of_week = 1

        if date == day:
            break
        else:
            day_of_week += 1

    return str(week_number)

def to_weekday(datestr):
    struct = time.strptime(datestr, "%Y-%m-%d %H:%M:%S")
    date = datetime.date(struct.tm_year,struct.tm_mon,struct.tm_mday)
    return date.isoweekday()

def daterange(date1_str, date2_str):
    start_date = to_date(date1_str + ' 00:00:00')
    end_date = to_date(date2_str + ' 00:00:00')
    if start_date > end_date:
        raise ValueError('start date cannot be after the end date.')
    for n in range(int((end_date - start_date).days) + 1):
        yield str(start_date + timedelta(n))
            
def date_range_condition_values(start_date, end_date):
    dates = daterange(start_date, end_date)
    for idx, date in enumerate(dates):
            yield date

df_decline_type = pd.read_csv(WORK_DIR + 'Decline_Type.csv')

msg_group = { 'declined' : 'decline', 
             'do_not_honor' : 'do not honor', 
             'txn_refused' : 'refuse', 
             'attempt_lower_amount' : 'lower amount',
            'Insufficient Funds' : 'insufficient',
            'not_allowed' : 'not allowed',
            'correct_cc_retry' : 'correct card',
            'invalid_cc' : 'invalid card',
            'lost_stolen' : 'lost or stolen',
            'invalid_account' : 'invalid account',
            'do_not_try' : 'do not try',
            'expired_card' : 'expired',
            'pickup_card' : 'pick',
            'blocked_first_used' : 'blocked',
            'invalid_txn' : 'invalid trans',
            'restricted_card' : 'restricted',
            'not_permitted' : 'not permitted',
            'expired card' : 'expired card',
            'unable to determine format' : 'determine format',
            'system error' : 'error',
            'no reply' : 'no reply',
             'no charge model found' : 'no charge model found',
             'issuer unavailable' : 'issuer unavailable',
             'litle http response code' : 'litle http response code',
             'corrupt input data to server' : 'corrupt input data to server',
             'ioexception' : 'ioexception',
             'invalid merchant' : 'invalid merchant',
            }

def group_response_msg(msg):
    other = 'Base'
    if isinstance(msg, str) == False:
        return other
    
    msg_lower = msg.lower()
    for key, val in msg_group.items():
        if val in msg_lower:
            return key
        
    return other   



# Merge failed attempt with the next attempt
def merge_by_sub(group):

    
    #Assign default values to additonal columns
    group['failed_attempt_date'] = ''
    group['failed_day_of_month'] = 0
    group['failed_response_code'] = ''
    group['failed_response_message'] = ''
    group['failed_decline_type'] = ''
    group['days_between'] = 0
    group['failed_event_uuid'] = ''    
    new_group = pd.DataFrame(columns = group.columns)
    
    sorted_group = group.sort_values(by=['transaction_date_in_string', 'cid'])
    initial_att_num = sorted_group.head(1)['renew_att_num'].iloc[0]
    last_att_num = sorted_group.tail(1)['renew_att_num'].iloc[0]
    if not last_att_num or not initial_att_num:
        group['event_uuid'] = '' 
        return group
#         return group.iloc[0]
        
    for att_num in range(int(initial_att_num), int(last_att_num)):
        next_att_num = int(att_num) + 1
        initial_att_indexes = group[group['renew_att_num'] == str(att_num)].index.tolist()
        next_att_indexes = group[group['renew_att_num'] == str(next_att_num)].index.tolist()
        
        for item in itertools.product(*[initial_att_indexes, next_att_indexes]):
            merged = group.loc[item[1]].copy(deep=True)
            inital_row = group.loc[item[0]]                   
            failed_attempt_date = inital_row['transaction_date_in_string']                   
            merged['failed_attempt_date'] = failed_attempt_date

            merged['failed_day_of_month'] = inital_row['day_of_month']
            merged['day_of_month'] = int(merged['day_of_month'])
            merged['failed_response_code'] = inital_row['response_code']
            
            failed_response_msg = inital_row['response_message']
            merged['failed_response_message'] = failed_response_msg  

            merged['failed_decline_type'] = decline_type(inital_row['response_message']) 
            merged['days_between'] = days_between_ds(merged)
            merged['failed_event_uuid'] =   inital_row['event_uuid']   
            new_group = new_group.append(merged)
            
           
    if new_group.empty:
        group['event_uuid'] = '' 
        return group
#         return group.iloc[0]
    
    return new_group


def decline_type(response_msg):
    '''Converts to decline_type based on the given response_msg'''
    dec_type = df_decline_type[df_decline_type['DECLINE_TEXT'] == response_msg]['DECLINE_TYPE']
    if dec_type.empty or dec_type.iloc[0] == 'Base' :
        return group_response_msg(response_msg)
    else:
        return dec_type.iloc[0]

#Convert from str to datetime
def to_date(datestr):
    struct = time.strptime(datestr, "%Y-%m-%d %H:%M:%S")
    date = datetime.date(struct.tm_year,struct.tm_mon,struct.tm_mday)
    return date

#Get difference between d2 and d1 in days.
def days_between(d1, d2):
    print('d1: {}, d2: {}'.format(d1, d2))
    d1 = to_date(d1)
    d2 = to_date(d2)
    print('after days_between')
    return abs((d2 - d1).days)

#Get difference between d2 and d1 in days.
def days_between_ds(df):
    d1 = to_date(df['failed_attempt_date'])
    d2 = to_date(df['transaction_date_in_string'])
    return abs((d2 - d1).days)

    
udfValueToStatus = udf(convert_status, IntegerType())  
# udfValueToStatus = udf(convert_status, StringType())  

createDDL = """CREATE OR REPLACE TEMPORARY VIEW cpg_transaction_history
     USING org.apache.spark.sql.cassandra
     OPTIONS (
     table "cpg_transaction_history",
     keyspace "dev_sarsks",
     cluster "MLP Cluster",
     pushdown "true")"""
spark.sql(createDDL)

df = spark.sql("SELECT * FROM cpg_transaction_history WHERE received_date like '2018-12-%' ")
# df = spark.sql("SELECT * FROM cpg_transaction_history WHERE received_date like '2018-01-%' or received_date like '2018-02-%' or received_date like '2018-03-%' or received_date like '2018-04-%' or received_date like '2018-05-%' or received_date like '2018-06-%' or received_date like '2018-07-%' or received_date like '2018-08-%' or received_date like '2018-09-%' or received_date like '2018-10-%' ")

sdf =  df\
    .filter("!(renew_att_num == '1' and new_status in ('Completed') and cid is null and status == 'New')")\
    .withColumn("success", udfValueToStatus("new_status"))\
    .withColumn("day_of_month", df["day_of_month"].cast(IntegerType()))\
    .withColumn("day_of_week", lower(col("day_of_week")))\
    .cache()

print(sdf.schema)
    
grouped_schema = sdf.schema.add("failed_attempt_date", StringType())\
                            .add("failed_day_of_month", IntegerType())\
                            .add("failed_response_code", StringType())\
                            .add("failed_response_message", StringType())\
                            .add("failed_decline_type", StringType())\
                            .add("days_between", IntegerType())\
                            .add("failed_event_uuid", StringType())
                            
                            
@pandas_udf(grouped_schema, PandasUDFType.GROUPED_MAP)
def merge_by_sub_spark(spark_df):
    pdf = pd.DataFrame(spark_df)
    result = pdf.groupby(['subscription_id', 'subsegment_id']).apply(merge_by_sub)
    return result                            
                            
time_start = time.time()

def _map_to_pandas(rdds):
    """ Needs to be here due to pickling issues """
    return [pd.DataFrame(list(rdds))]

def toPandas(df, n_partitions=None):
    """
    Returns the contents of `df` as a local `pandas.DataFrame` in a speedy fashion. The DataFrame is
    repartitioned if `n_partitions` is passed.
    :param df:              pyspark.sql.DataFrame
    :param n_partitions:    int or None
    :return:                pandas.DataFrame
    """
    if n_partitions is not None: df = df.repartition(n_partitions)
    df_pand = df.rdd.mapPartitions(_map_to_pandas).collect()
    df_pand = pd.concat(df_pand)
    df_pand.columns = df.columns
    return df_pand

sdf_filtered = sdf\
        .filter(sdf.renew_att_num.isNotNull())\
        .groupby("subscription_id", "subsegment_id")\
        .apply(merge_by_sub_spark)\
        .filter("event_uuid != ''")  #.toPandas()

# pdf_all = toPandas(sdf_filtered, 128)  
pdf_all = sdf_filtered.toPandas()
print("# total time:", time.time() - time_start)      # total time: 2364.714904308319
    
pdf_all.shape

StructType(List(StructField(received_date,DateType,true),StructField(site_id,StringType,true),StructField(subscription_id,StringType,true),StructField(transaction_id,StringType,true),StructField(transaction_date,TimestampType,true),StructField(added_expiry_years,StringType,true),StructField(bank_code,StringType,true),StructField(bank_name,StringType,true),StructField(billing_country,StringType,true),StructField(bin,StringType,true),StructField(card_brand,StringType,true),StructField(card_category,StringType,true),StructField(card_class,StringType,true),StructField(card_usage,StringType,true),StructField(cc_expiration_date,StringType,true),StructField(cid,StringType,true),StructField(date_increment,StringType,true),StructField(day_of_month,IntegerType,true),StructField(day_of_week,StringType,true),StructField(event_uuid,StringType,true),StructField(exp_setting_id,StringType,true),StructField(funding_source,StringType,true),StructField(issuer_country,StringType,true),StructField(merchant

(1673882, 53)

In [67]:
# pdf_all.to_csv(WORK_DIR + 'retry_success_next_attempt_1_to_10.csv')
# pdf_all.to_csv(WORK_DIR + 'retry_success_next_attempt_12.csv')
# pdf_all.to_csv(WORK_DIR + 'retry_success_by_calendar_attempt_01_to_12.csv')

In [9]:
from IPython.display import display
pd.options.display.max_columns = None
pdf_all.head()

,received_date,site_id,subscription_id,transaction_id,transaction_date,added_expiry_years,bank_code,bank_name,billing_country,bin,card_brand,card_category,card_class,card_usage,cc_expiration_date,cid,date_increment,day_of_month,day_of_week,event_uuid,exp_setting_id,funding_source,issuer_country,merchant_descriptor,merchant_number,mid_entity_code,new_status,order_entity_code,payment_amount,payment_amount_usd,payment_currency,payment_method_id,payment_service_id,renew_att_num,request_amount,request_amount_usd,request_currency,requisition_id,response_code,response_message,status,subsegment_id,transaction_date_in_string,transaction_hour,transaction_type,success,failed_attempt_date,failed_day_of_month,failed_response_code,failed_response_message,failed_decline_type,days_between,failed_event_uuid
0,2018-11-25,adbehap,10001242501,7804763323,2018-11-25 06:35:55,,NON3DS,Banco Popular y de Desarrollo Comunal,CR,491261,Visa,Visa Classic,Consumer,Debit - NON USA Consumer Debit - No Pin Access,0222,,,25,sunday,789c8475-73d7-4f18-9ea0-1d99b7027a1b,,Debit,CR,DRI*Adobe Sales,1411163460,DR_IRELAND-ENTITY,Completed,DR_IRELAND-ENTITY,9.99,9.99,USD,Visa,netgiro-bms,2,9.99,9.99,USD,54674840200,0,SUCCESS[00] [021570] [111800] [111800],New,17629617200,2018-11-25 00:35:55,00,Authorize,1,2018-11-18 00:35:57,18,27083,Insufficient Funds[51] [] [111800] [111800],,0,4caca341-1b41-4031-a1ba-2d522eee10da
1,2018-06-25,adbeheu,10001252801,6960360213,2018-06-25 06:19:34,,NON3DS,QNB Finansbank A.S.,TR,498749,Visa,Visa Classic,Consumer,NON USA Consumer Debit - No Pin Access,0420,,,25,monday,b2d6c31a-ecc1-477c-b2fb-435da1488182,,Debit,TR,DRI*Adobe Systems,1156962062,DR_IRELAND-ENTITY,Completed,DR_IRELAND-ENTITY,103.62,22.089822,TRY,Visa,netgiro-seb,2,103.62,22.089822,TRY,49300935300,0,SUCCESS[00] [854564] [] [],New,16248444700,2018-06-25 01:19:34,01,Authorize,1,2018-06-18 01:18:03,18,27083,Insufficient Funds[51] [] [111800] [111800],,0,9c0891e7-79d3-403c-8b53-26ff28bef04e
2,2018-04-13,avast,10001280501,6662963373,2018-04-13 01:12:39,,NON3DS,Standard Chartered Bank Tanzania Limited,TZ,478680,Visa,Visa Gold,Consumer,NON USA Consumer Debit - No Pin Access,0322,,,13,friday,f350d6f2-f04c-4167-a25e-1c76eaea3b83,,Debit,TZ,DRI*AVAST Software,1411163460,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,79.99,79.99,USD,Visa,netgiro-bms,2,79.99,79.99,USD,46803884300,27050,Do not try again/use alternate payment card.[0...,New,13258980000,2018-04-13 01:12:39,01,Authorize,0,2018-04-03 01:07:59,3,27050,Do not try again/use alternate payment card.[0...,,0,f89a7e5d-688a-4021-af0a-d553519799fd
3,2018-04-18,avast,10001280501,6681738323,2018-04-18 01:21:29,,NON3DS,Standard Chartered Bank Tanzania Limited,TZ,478680,Visa,Visa Gold,Consumer,NON USA Consumer Debit - No Pin Access,0322,,,18,wednesday,673671cc-cc57-400c-b872-8ff9efb09a6d,,Debit,TZ,DRI*AVAST Software,1411163460,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,79.99,79.99,USD,Visa,netgiro-bms,3,79.99,79.99,USD,46971082700,27050,Do not try again/use alternate payment card.[0...,New,13258980000,2018-04-18 01:21:29,01,Authorize,0,2018-04-13 01:12:39,13,27050,Do not try again/use alternate payment card.[0...,,0,f350d6f2-f04c-4167-a25e-1c76eaea3b83
4,2018-04-13,avastbr,10001441601,6662976133,2018-04-13 01:14:17,,NON3DS,ITAU UNIBANCO S.A.,BR,539083,MasterCard,(Unembossed) Standard MasterCard Card,Consumer,True credit (No PIN/Signature capability),1223,,,13,friday,05fa2932-0183-4a05-90b9-eb2a54043dda,,Credit,BR,Avast Brazil,1883448942,DR_BRAZIL-ENTITY,Declined,DR_BRAZIL-ENTITY,99,29.409792,BRL,MasterCard,netgiro-br,2,99,29.409792,BRL,46804061000,27001,Transaction refused[62] [null] [] [],New,13259159000,2018-04-13 01:14:17,01,Authorize,0,2018-04-08 01:03:36,8,27001,Transaction refused[62] [null] [] [],,0,886c6b02-897e-4ace-9ae0-c820bc80cc0e


In [8]:
# Merge only the last two renew_att_num with the last single attempt
import time

import pandas as pd


from pyspark.sql.types import *
from pyspark.sql.functions import col, count, rand, collect_list, explode, struct, count, lit, lower
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.functions import udf
from pyspark.sql.types import *
import pyspark_cassandra
        
import calendar, datetime
from datetime import timedelta

WORK_DIR = '/var/spark/ml_files/'


def convert_status(value):
    if value == 'Completed':
        return 1
    else:
        return 0
    
    
def cast_to_int(value, default=-1):
    try:
        return int(value)
    except (ValueError, TypeError):
        return default


#Convert from str to datetime
def to_date(datestr):
    struct = time.strptime(datestr, "%Y-%m-%d %H:%M:%S")
    date = datetime.date(struct.tm_year,struct.tm_mon,struct.tm_mday)
    return date

#Get difference between d2 and d1 in days.
def days_between(d1, d2):
    d1 = to_date(d1)
    d2 = to_date(d2)
    return abs((d2 - d1).days)

#Determines the week (number) of the month
def week_of_month(datestr):
    date = to_date(datestr)
    #Calendar object. 6 = Start on Sunday, 0 = Start on Monday
    cal_object = calendar.Calendar(6)
    month_calendar_dates = cal_object.itermonthdates(date.year,date.month)

    day_of_week = 1
    week_number = 1

    for day in month_calendar_dates:
        #add a week and reset day of week
        if day_of_week > 7:
            week_number += 1
            day_of_week = 1

        if date == day:
            break
        else:
            day_of_week += 1

    return str(week_number)

def to_weekday(datestr):
    struct = time.strptime(datestr, "%Y-%m-%d %H:%M:%S")
    date = datetime.date(struct.tm_year,struct.tm_mon,struct.tm_mday)
    return date.isoweekday()

def daterange(date1_str, date2_str):
    start_date = to_date(date1_str + ' 00:00:00')
    end_date = to_date(date2_str + ' 00:00:00')
    if start_date > end_date:
        raise ValueError('start date cannot be after the end date.')
    for n in range(int((end_date - start_date).days) + 1):
        yield str(start_date + timedelta(n))
            
def date_range_condition_values(start_date, end_date):
    dates = daterange(start_date, end_date)
    for idx, date in enumerate(dates):
            yield date

df_decline_type = pd.read_csv(WORK_DIR + 'Decline_Type.csv')

msg_group = { 'declined' : 'decline', 
             'do_not_honor' : 'do not honor', 
             'txn_refused' : 'refuse', 
             'attempt_lower_amount' : 'lower amount',
            'Insufficient Funds' : 'insufficient',
            'not_allowed' : 'not allowed',
            'correct_cc_retry' : 'correct card',
            'invalid_cc' : 'invalid card',
            'lost_stolen' : 'lost or stolen',
            'invalid_account' : 'invalid account',
            'do_not_try' : 'do not try',
            'expired_card' : 'expired',
            'pickup_card' : 'pick',
            'blocked_first_used' : 'blocked',
            'invalid_txn' : 'invalid trans',
            'restricted_card' : 'restricted',
            'not_permitted' : 'not permitted',
            'expired card' : 'expired card',
            'unable to determine format' : 'determine format',
            'system error' : 'error'
            }

def group_response_msg(msg):
    other = 'Base'
    if isinstance(msg, str) == False:
        return other
    
    msg_lower = msg.lower()
    for key, val in msg_group.items():
        if val in msg_lower:
            return key
        
    return other   



# Merge only the last two renew_att_num with the last single attempt
def merge_by_sub(group):

    
    #Assign default values to additonal columns
    group['failed_attempt_date'] = ''
    group['failed_day_of_month'] = 0
    group['failed_response_code'] = ''
    group['failed_response_message'] = ''
    group['failed_decline_type'] = ''
    group['days_between'] = 0
    group['failed_event_uuid'] = ''    
    
    last = pd.DataFrame() 
    df_success = group[group['success'] == 1]

    if not df_success.empty:
        df_last_attempt = df_success.iloc[0]
    else:
        sorted_group = group.sort_values(by=['transaction_date_in_string', 'cid'])
        df_last_fail = sorted_group.tail(1)
        if not df_last_fail.empty:
            df_last_attempt =  df_last_fail.iloc[0]
        else:
            group['event_uuid'] = '' 
            return group
    
    if df_last_attempt['renew_att_num']:
        last_att_num = int(df_last_attempt['renew_att_num'])
        previous_att_num = last_att_num - 1
    else:
        group['event_uuid'] = '' 
        return group
   
    df_failed_txn = group[(group['renew_att_num'].isin([ str(previous_att_num),  df_last_attempt['renew_att_num'] ])) & (group['event_uuid'] != df_last_attempt['event_uuid']) ]
    if df_failed_txn.empty:
        group['event_uuid'] = '' 
        return group
    
    last = last.append( [df_last_attempt] * len(df_failed_txn),ignore_index=True ).set_index(df_failed_txn.index)
        
    failed_attempt_date = df_failed_txn['transaction_date_in_string']
    
    last['failed_attempt_date'] = failed_attempt_date
    last['failed_day_of_month'] = df_failed_txn['day_of_month']
    last['day_of_month'] = last['day_of_month'].astype(int)
    last['failed_response_code'] = df_failed_txn['response_code']
    
    failed_response_msg = df_failed_txn['response_message']
    last['failed_response_message'] = failed_response_msg  
    
    last['failed_decline_type'] = df_failed_txn['response_message'].apply(decline_type) 
    last['days_between'] = last.apply(days_between_ds, axis=1)    
    last['failed_event_uuid'] =   df_failed_txn['event_uuid']   
        
    return last


def decline_type(response_msg):
    '''Converts to decline_type based on the given response_msg'''
    dec_type = df_decline_type[df_decline_type['DECLINE_TEXT'] == response_msg]['DECLINE_TYPE']
    if dec_type.empty or dec_type.iloc[0] == 'Base' :
        return group_response_msg(response_msg)
    else:
        return dec_type.iloc[0]

#Convert from str to datetime
def to_date(datestr):
    struct = time.strptime(datestr, "%Y-%m-%d %H:%M:%S")
    date = datetime.date(struct.tm_year,struct.tm_mon,struct.tm_mday)
    return date

#Get difference between d2 and d1 in days.
def days_between(d1, d2):
    d1 = to_date(d1)
    d2 = to_date(d2)

    return abs((d2 - d1).days)

#Get difference between d2 and d1 in days.
def days_between_ds(df):
    d1 = to_date(df['failed_attempt_date'])
    d2 = to_date(df['transaction_date_in_string'])
    return abs((d2 - d1).days)

    
udfValueToStatus = udf(convert_status, IntegerType())  

createDDL = """CREATE OR REPLACE TEMPORARY VIEW cpg_transaction_history
     USING org.apache.spark.sql.cassandra
     OPTIONS (
     table "cpg_transaction_history",
     keyspace "dev_sarsks",
     cluster "MLP Cluster",
     pushdown "true")"""
spark.sql(createDDL)

df = spark.sql("SELECT * FROM cpg_transaction_history WHERE received_date like '2019-01-%' ")
# df = spark.sql("SELECT * FROM cpg_transaction_history WHERE received_date like '2018-01-%' or received_date like '2018-02-%' or received_date like '2018-03-%' or received_date like '2018-04-%' or received_date like '2018-05-%' or received_date like '2018-06-%' or received_date like '2018-07-%' or received_date like '2018-08-%' or received_date like '2018-09-%' or received_date like '2018-10-%' or received_date like '2018-11-%' or received_date like '2018-12-%' ")

sdf =  df\
    .filter("!(renew_att_num == '1' and new_status in ('Completed') and cid is null and status == 'New')")\
    .withColumn("success", udfValueToStatus("new_status"))\
    .withColumn("day_of_month", df["day_of_month"].cast(IntegerType()))\
    .withColumn("day_of_week", lower(col("day_of_week")))\
    .cache()

print(sdf.schema)
    
grouped_schema = sdf.schema.add("failed_attempt_date", StringType())\
                            .add("failed_day_of_month", IntegerType())\
                            .add("failed_response_code", StringType())\
                            .add("failed_response_message", StringType())\
                            .add("failed_decline_type", StringType())\
                            .add("days_between", IntegerType())\
                            .add("failed_event_uuid", StringType())
                            
                            
@pandas_udf(grouped_schema, PandasUDFType.GROUPED_MAP)
def merge_by_sub_spark(spark_df):
    pdf = pd.DataFrame(spark_df)
    result = pdf.groupby(['subscription_id', 'subsegment_id']).apply(merge_by_sub)
    return result                            
                            
time_start = time.time()

def _map_to_pandas(rdds):
    """ Needs to be here due to pickling issues """
    return [pd.DataFrame(list(rdds))]

def toPandas(df, n_partitions=None):
    """
    Returns the contents of `df` as a local `pandas.DataFrame` in a speedy fashion. The DataFrame is
    repartitioned if `n_partitions` is passed.
    :param df:              pyspark.sql.DataFrame
    :param n_partitions:    int or None
    :return:                pandas.DataFrame
    """
    if n_partitions is not None: df = df.repartition(n_partitions)
    df_pand = df.rdd.mapPartitions(_map_to_pandas).collect()
    df_pand = pd.concat(df_pand)
    df_pand.columns = df.columns
    return df_pand

sdf_filtered = sdf\
        .filter(sdf.renew_att_num.isNotNull())\
        .groupby("subscription_id", "subsegment_id")\
        .apply(merge_by_sub_spark)\
        .filter("event_uuid != ''")  #.toPandas()

pdf_all = toPandas(sdf_filtered, 64)  
# pdf_all = sdf_filtered.toPandas()
print("# total time:", time.time() - time_start)      # total time: 2364.714904308319
    
pdf_all.shape

StructType(List(StructField(received_date,DateType,true),StructField(site_id,StringType,true),StructField(subscription_id,StringType,true),StructField(transaction_id,StringType,true),StructField(transaction_date,TimestampType,true),StructField(added_expiry_years,StringType,true),StructField(bank_code,StringType,true),StructField(bank_name,StringType,true),StructField(billing_country,StringType,true),StructField(bin,StringType,true),StructField(card_brand,StringType,true),StructField(card_category,StringType,true),StructField(card_class,StringType,true),StructField(card_usage,StringType,true),StructField(cc_expiration_date,StringType,true),StructField(cid,StringType,true),StructField(date_increment,StringType,true),StructField(day_of_month,IntegerType,true),StructField(day_of_week,StringType,true),StructField(event_uuid,StringType,true),StructField(exp_setting_id,StringType,true),StructField(funding_source,StringType,true),StructField(issuer_country,StringType,true),StructField(merchant

Py4JJavaError: An error occurred while calling o1625.javaToPython.
: org.apache.spark.sql.catalyst.errors.package$TreeNodeException: execute, tree:
Exchange RoundRobinPartitioning(64)
+- *(3) Filter (isnotnull(event_uuid#2359) && NOT (event_uuid#2359 = ))
   +- FlatMapGroupsInPandas [subscription_id#1787, subsegment_id#1826], merge_by_sub_spark(received_date#1785, site_id#1786, subscription_id#1787, transaction_id#1788, transaction_date#1789, added_expiry_years#1790, bank_code#1791, bank_name#1792, billing_country#1793, bin#1794, card_brand#1795, card_category#1796, card_class#1797, card_usage#1798, cc_expiration_date#1799, cid#1800, date_increment#1801, day_of_month#1968, day_of_week#2015, event_uuid#1804, exp_setting_id#1805, funding_source#1806, issuer_country#1807, merchant_descriptor#1808, merchant_number#1809, mid_entity_code#1810, new_status#1811, order_entity_code#1812, payment_amount#1813, payment_amount_usd#1814, payment_currency#1815, payment_method_id#1816, payment_service_

In [77]:
from IPython.display import display
pd.options.display.max_columns = None
pdf_all_07[pdf_all_07['subscription_id'] == '10363932201'].sort_values(by=['renew_att_num'])

,received_date,site_id,subscription_id,transaction_id,transaction_date,added_expiry_years,bank_code,bank_name,billing_country,bin,card_brand,card_category,card_class,card_usage,cc_expiration_date,cid,date_increment,day_of_month,day_of_week,event_uuid,exp_setting_id,funding_source,issuer_country,merchant_descriptor,merchant_number,mid_entity_code,new_status,order_entity_code,payment_amount,payment_amount_usd,payment_currency,payment_method_id,payment_service_id,renew_att_num,request_amount,request_amount_usd,request_currency,requisition_id,response_code,response_message,status,subsegment_id,transaction_date_in_string,transaction_hour,transaction_type,success
29368,2018-07-10,avgstore,10363932201,7034206383,2018-07-10 06:44:58,,NON3DS,Foreningen af Danske Kortudstedere F.M.B.A,DK,457105,Visa,Visa Classic,Consumer,NON USA Consumer Debit - No Pin Access,0120,,,10,Tuesday,f25a2734-918a-4725-933f-031a358efa2f,,Debit,DK,DRI*AVG Technologies,1156962062,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,549,86.835015,DKK,Visa,netgiro-seb,1,549,86.835015,DKK,49782722000,27001,Transaction refused[05] [000000] [] [],New,13865645800,2018-07-10 01:44:58,01,Authorize,0.0
28528,2018-07-10,avgstore,10363932201,7034206523,2018-07-10 06:44:59,,NON3DS,Foreningen af Danske Kortudstedere F.M.B.A,DK,457105,Visa,Visa Classic,Consumer,NON USA Consumer Debit - No Pin Access,0120,RETRY_DECLINED.2,,10,Tuesday,7a9e2bdc-3828-48cf-b956-5869d250fa72,,Debit,DK,DRI*AVG Technologies,1411163460,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,549,86.835015,DKK,Visa,netgiro-bms,1,549,86.835015,DKK,49782722000,27001,Transaction refused[05] [] [111800] [111800],New,13865645800,2018-07-10 01:44:59,01,Authorize,0.0
27718,2018-07-18,avgstore,10363932201,7074667233,2018-07-18 06:36:50,,NON3DS,Foreningen af Danske Kortudstedere F.M.B.A,DK,457105,Visa,Visa Classic,Consumer,NON USA Consumer Debit - No Pin Access,0120,,,18,Wednesday,3f45c21c-df43-4dd2-8068-a045f1283cff,,Debit,DK,DRI*AVG Technologies,1156962062,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,549,86.224081,DKK,Visa,netgiro-seb,2,549,86.224081,DKK,50032013300,27001,Transaction refused[05] [000000] [] [],New,13865645800,2018-07-18 01:36:50,01,Authorize,0.0
20439,2018-07-18,avgstore,10363932201,7074667383,2018-07-18 06:36:52,,NON3DS,Foreningen af Danske Kortudstedere F.M.B.A,DK,457105,Visa,Visa Classic,Consumer,NON USA Consumer Debit - No Pin Access,0120,RETRY_DECLINED.2,,18,Wednesday,e210e36f-737c-40a1-ae78-0bffad118468,,Debit,DK,DRI*AVG Technologies,1411163460,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,549,86.224081,DKK,Visa,netgiro-bms,2,549,86.224081,DKK,50032013300,27001,Transaction refused[05] [] [111800] [111800],New,13865645800,2018-07-18 01:36:52,01,Authorize,0.0
13,2018-07-25,avgstore,10363932201,7109736923,2018-07-25 06:28:30,,NON3DS,Foreningen af Danske Kortudstedere F.M.B.A,DK,457105,Visa,Visa Classic,Consumer,NON USA Consumer Debit - No Pin Access,0120,RETRY_DECLINED.2,,25,Wednesday,7159bc5a-7e30-4ce3-9183-957c7c93c66e,,Debit,DK,DRI*AVG Technologies,1411163460,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,549,86.258375,DKK,Visa,netgiro-bms,3,549,86.258375,DKK,50252786700,27001,Transaction refused[05] [] [111800] [111800],New,13865645800,2018-07-25 01:28:30,01,Authorize,0.0
7675,2018-07-25,avgstore,10363932201,7109736833,2018-07-25 06:28:29,,NON3DS,Foreningen af Danske Kortudstedere F.M.B.A,DK,457105,Visa,Visa Classic,Consumer,NON USA Consumer Debit - No Pin Access,0120,,,25,Wednesday,db145677-1fa3-4b42-9893-0b65f0db58e4,,Debit,DK,DRI*AVG Technologies,1156962062,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,549,86.258375,DKK,Visa,netgiro-seb,3,549,86.258375,DKK,50252786700,27001,Transaction refused[05] [000000] [] [],New,13865645800,2018-07-25 01:28:29,01,Authorize,0.0


In [78]:
pdf_all_07[pdf_all_07['subscription_id'] == '10363932201'].groupby(["subscription_id", "subsegment_id"]).apply(merge_by_sub)
# pdf_all_07[pdf_all_07['subscription_id'] == '11623213001'].groupby(['subscription_id', 'subsegment_id']).apply(merge_by_sub)

received_date   site_id subscription_id  \
subscription_id subsegment_id                                                 
10363932201     13865645800   27718    2018-07-25  avgstore     10363932201   
                              20439    2018-07-25  avgstore     10363932201   
                              7675     2018-07-25  avgstore     10363932201   

                                    transaction_id    transaction_date  \
subscription_id subsegment_id                                            
10363932201     13865645800   27718     7109736923 2018-07-25 06:28:30   
                              20439     7109736923 2018-07-25 06:28:30   
                              7675      7109736923 2018-07-25 06:28:30   

                                    added_expiry_years bank_code  \
subscription_id subsegment_id                                      
10363932201     13865645800   27718                       NON3DS   
                              20439                       NON3DS   
                              7675                        NON3DS   

                                                                      bank_name  \
subscription_id subsegment_id                                                     
10363932201     13865645800   27718  Foreningen af Danske Kortudstedere F.M.B.A   
                              20439  Foreningen af Danske Kortudstedere F.M.B.A   
                              7675   Foreningen af Danske Kortudstedere F.M.B.A   

                                    billing_country     bin card_brand  \
subscription_id subsegment_id                                            
10363932201     13865645800   27718              DK  457105       Visa   
                              20439              DK  457105       Visa   
                              7675               DK  457105       Visa   

                                    card_category card_class  \
subscription_id subsegment_id                                  
10363932201     13865645800   27718  Visa Classic   Consumer   
                              20439  Visa Classic   Consumer   
                              7675   Visa Classic   Consumer   

                                                                 card_usage  \
subscription_id subsegment_id                                                 
10363932201     13865645800   27718  NON USA Consumer Debit - No Pin Access   
                              20439  NON USA Consumer Debit - No Pin Access   
                              7675   NON USA Consumer Debit - No Pin Access   

                                    cc_expiration_date               cid  \
subscription_id subsegment_id                                              
10363932201     13865645800   27718               0120  RETRY_DECLINED.2   
                              20439               0120  RETRY_DECLINED.2   
                              7675                0120  RETRY_DECLINED.2   

                                    date_increment  day_of_month day_of_week  \
subscription_id subsegment_id                                                  
10363932201     13865645800   27718                           25   Wednesday   
                              20439                           25   Wednesday   
                              7675                            25   Wednesday   

                                                               event_uuid  \
subscription_id subsegment_id                                               
10363932201     13865645800   27718  7159bc5a-7e30-4ce3-9183-957c7c93c66e   
                              20439  7159bc5a-7e30-4ce3-9183-957c7c93c66e   
                              7675   7159bc5a-7e30-4ce3-9183-957c7c93c66e   

                                    exp_setting_id funding_source  \
subscription_id subsegment_id                                       
10363932201     13865645800   27718                         Debit   
                              20439              

In [13]:
pdf_all_07['success'] = pdf_all_07['new_status'].map({'Completed':1,'Declined':0, 'Failed':0, 'Reversed':0})

In [40]:
def merge_by_sub(group):

    
    #Assign default values to additonal columns
    group['failed_attempt_date'] = ''
    group['failed_day_of_month'] = 0
    group['failed_response_code'] = ''
    group['failed_response_message'] = ''
    group['failed_decline_type'] = ''
    group['days_between'] = 0
    group['failed_event_uuid'] = ''    
    
    
    
    last = pd.DataFrame() 
    df_success = group[group['success'] == 1]

    if not df_success.empty:
        df_last_attempt = df_success.iloc[0]
    else:
        sorted_group = group.sort_values(by=['transaction_date_in_string', 'cid'])
        df_last_fail = sorted_group.tail(1)
        if not df_last_fail.empty:
            df_last_attempt =  df_last_fail.iloc[0]
        else:
            group['event_uuid'] = '' 
            return group
    
    
    df_failed_txn = group[group['event_uuid'] != df_last_attempt['event_uuid']]
    if df_failed_txn.empty:
        group['event_uuid'] = '' 
        return group
    
    last = last.append( [df_last_attempt] * len(df_failed_txn),ignore_index=True ).set_index(df_failed_txn.index)
        
    failed_attempt_date = df_failed_txn['transaction_date_in_string']
    
    last['failed_attempt_date'] = failed_attempt_date
    last['failed_day_of_month'] = df_failed_txn['day_of_month']
    last['day_of_month'] = last['day_of_month'].astype(int)
    last['failed_response_code'] = df_failed_txn['response_code']
    
    failed_response_msg = df_failed_txn['response_message']
    last['failed_response_message'] = failed_response_msg  
    
#     last['failed_decline_type'] = df_failed_txn['response_message'].apply(decline_type) 
#     last['days_between'] = last.apply(days_between_ds, axis=1)    
    last['failed_event_uuid'] =   df_failed_txn['event_uuid']   
        
    return last

In [36]:
pdf_all_07['success'] = pdf_all_07['new_status'].map({'Completed':1,'Declined':0, 'Failed':0, 'Reversed':0})

In [65]:
pdf_all_07[pdf_all_07['subscription_id'] == '1714323609'].groupby(['subscription_id', 'subsegment_id']).apply(merge_by_sub)

,,,received_date,site_id,subscription_id,transaction_id,transaction_date,added_expiry_years,bank_code,bank_name,billing_country,bin,card_brand,card_category,card_class,card_usage,cc_expiration_date,cid,date_increment,day_of_month,day_of_week,event_uuid,exp_setting_id,funding_source,issuer_country,merchant_descriptor,merchant_number,mid_entity_code,new_status,order_entity_code,payment_amount,payment_amount_usd,payment_currency,payment_method_id,payment_service_id,renew_att_num,request_amount,request_amount_usd,request_currency,requisition_id,response_code,response_message,status,subsegment_id,transaction_date_in_string,transaction_hour,transaction_type,success,failed_attempt_date,failed_day_of_month,failed_response_code,failed_response_message,failed_decline_type,days_between,failed_event_uuid
subscription_id,subsegment_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1714323609,13715900300,25450,2018-07-27,kasperde,1714323609,7121207423,2018-07-27 12:52:37,,NON3DS,VISECA CARD SERVICES S.A.,CH,540448,MasterCard,MasterCard World Card,Consumer,True credit (No PIN/Signature capability),0120,,,27,Friday,f544aa04-2617-44c4-bea9-7f81beade6b6,,Credit,CH,DRI*Kaspersky Lab,1411163460,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,39.95,46.80542,EUR,MasterCard,netgiro-bms,4,39.95,46.80542,EUR,50340338000,27050,Do not try again/use alternate payment card.[0...,New,13715900300,2018-07-27 07:52:37,07,Authorize,0.0,2018-07-13 05:37:32,13,27050,Do not try again/use alternate payment card.[0...,,0,b58d134d-cb95-460d-8dca-df9f2073dbb1


In [16]:
bin_profile = pd.read_csv(WORK_DIR + 'bin_profile.csv')
bin_profile.shape



(32272, 11)

In [17]:
bin_profile['bin'] = bin_profile['bin'].apply(str).str.replace('.0', '', regex=False)
bin_profile.head()

,Unnamed: 0,bin,Mean,Min,Max,StdDev,Max_95,Max_99,Min_1,Min_10,count
0,0,nan,18.043255,0.011639,410.000000,27.498705,79.410000,102.195200,4.590894,4.590894,3579
1,5,352818,19.118323,4.800000,51.479167,19.031746,45.503799,50.284093,4.800000,4.800000,7
2,7,352821,49.015144,48.102689,50.518630,0.988569,50.246824,50.464269,48.115695,48.115695,5
3,8,352822,34.316981,18.714051,49.360468,10.081940,46.126243,48.713623,18.714051,18.714051,10
4,12,352831,39.732697,18.336803,68.206773,16.005322,62.103159,66.986050,18.336803,18.336803,9


In [18]:
pdf_all = pd.merge(pdf_all, bin_profile[['bin','Mean','Min','Max','StdDev','Max_95','Max_99','Min_1']], left_on='bin', right_on='bin', how='left')
pdf_all.shape

(6605903, 66)

In [36]:
pdf_all_05 = pd.merge(pdf_all_05, bin_profile[['bin','Mean','Min','Max','StdDev','Max_95','Max_99','Min_1']], left_on='bin', right_on='bin', how='left')
pdf_all_05.shape

(536234, 60)

In [19]:
# pdf_all.to_csv(WORK_DIR + 'retry_success_last_two_with_bin.csv')
# pdf_all.to_csv(WORK_DIR + 'retry_success_last_two_with_bin_01_to_12.csv')
pdf_all.to_csv(WORK_DIR + 'retry_success_by_calendar_attempt_01_to_12.csv')

In [14]:
from IPython.display import display
pd.options.display.max_columns = None
pdf_all[ (pdf_all['transaction_date'] >= '2018-05-06') & (pdf_all['transaction_date'] <= '2018-05-20') & (pdf_all['failed_day_of_month'] == 5) ].head(5)

,received_date,site_id,subscription_id,transaction_id,transaction_date,added_expiry_years,bank_code,bank_name,billing_country,bin,card_brand,card_category,card_class,card_usage,cc_expiration_date,cid,date_increment,day_of_month,day_of_week,event_uuid,exp_setting_id,funding_source,issuer_country,merchant_descriptor,merchant_number,mid_entity_code,new_status,order_entity_code,payment_amount,payment_amount_usd,payment_currency,payment_method_id,payment_service_id,renew_att_num,request_amount,request_amount_usd,request_currency,requisition_id,response_code,response_message,status,subsegment_id,transaction_date_in_string,transaction_hour,transaction_type,success,failed_attempt_date,failed_day_of_month,failed_response_code,failed_response_message,failed_decline_type,days_between,failed_event_uuid,Mean,Min,Max,StdDev,Max_95,Max_99,Min_1
356,2018-05-12,kasperbr,10051863501,6770243363,2018-05-12 00:58:41,,NON3DS,ITAU UNIBANCO S.A.,BR,527496,MasterCard,(Unembossed) Standard MasterCard Card,Consumer,True credit (No PIN/Signature capability),0623,,,12,saturday,2a5db191-783a-4793-883c-177a97d59198,,Credit,BR,Kaspersky BR,1883448942,DR_BRAZIL-ENTITY,Declined,DR_BRAZIL-ENTITY,169.9,47.723641,BRL,MasterCard,netgiro-br,3,169.9,47.723641,BRL,47840177100,27001,Transaction refused[62] [null] [] [],New,13354010500,2018-05-12 00:58:41,00,Authorize,0,2018-05-05 01:11:49,5,27001,Transaction refused[62] [null] [] [],Hard Fraud,7,92d58d47-63ca-4d2b-a3cb-8315e4ef8a2e,21.978581,2.490000,367.306162,15.864276,47.400000,61.946008,9.030781
451,2018-05-10,avast,10070377501,6762459873,2018-05-10 04:01:22,,NON3DS,Barclays Bank PLC,GB,465858,Visa,Visa Classic,Consumer,NON USA Consumer Debit - No Pin Access,1119,,,10,thursday,4e4ad566-1d61-4dde-acaa-94d5a42ca946,,Debit,GB,DRI*AVAST Software,1850961065,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,59.99,81.493649,GBP,Visa,netgiro-bms,3,59.99,81.493649,GBP,47774937900,27050,Do not try again/use alternate payment card.[5...,New,13382730300,2018-05-10 04:01:22,04,Authorize,0,2018-05-05 01:16:10,5,27050,Do not try again/use alternate payment card.[5...,do_not_try,5,826c37ec-1e78-4cd9-8351-7f2bce19e2d4,47.138270,1.291424,853.281905,29.119373,88.800000,124.393270,6.450000
454,2018-05-10,kasper,10070837601,6761623203,2018-05-10 01:08:52,3,NON3DS,,DE,513143,,,,,1020,RETRY_DECLINED.2,3,10,thursday,dc10e465-47e5-41ab-8c7f-c5bad07103c0,2952811,,,DRI*Kaspersky Lab,1009072982,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,19.95,23.698605,EUR,MasterCard,drwp-fd,3,19.95,23.698605,EUR,47765604600,27001,Transaction refused[null] [null] [null] [null],New,13383137300,2018-05-10 01:08:52,01,Authorize,0,2018-05-05 01:17:46,5,27001,Transaction refused[05] [000000] [] [],txn_refused,5,75e1963d-40bc-4b0b-aac8-16045274393c,55.230222,2.490000,226.734066,25.429331,98.203723,121.941905,24.417005
459,2018-05-10,avast,10071184501,6761555553,2018-05-10 01:00:57,,DE12BE721F3C7000,Texas Bank and Trust Company,US,421778,Visa,Visa Classic,Consumer,Debit Hybrid (PIN and Signature),0919,,,10,thursday,49cc67f1-5269-47be-9610-046afe2f03b9,,Debit,US,DRI*AVAST Software,311009012882,DR_INC-ENTITY,Declined,DR_INC-ENTITY,64.94,64.94,USD,Visa,firstdata,3,64.94,64.94,USD,47764921600,201,14: INV ACCT NUM,New,13383454500,2018-05-10 01:00:57,01,Authorize,0,2018-05-05 01:11:25,5,201,301: Invalid Account Number,invalid_account,5,1d1eb830-f522-4fff-b545-a31a170f8ef2,68.803763,16.230000,116.860000,21.081258,108.240000,108.929600,48.700000
461,2018-05-10,avast,10071480101,6761583603,2018-05-10 01:04:20,,NON3DS,MULTIFINANZAS COMPANIA FINANCIERA S.A.,AR,550480,MasterCard,MasterCard Prepaid Card (Non-US),Consumer,True credit (No PIN/Signature capability),0420,RETRY_DECLINED.2,,10,thursday,377d7d27-5648-4ae6-897d-ad8ad0cb39f3,,Credit,AR,DRI*AVAST Software,1411163460,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,299,14.210979,ARS,MasterCard,netgiro-bms,3,299,14.210979,ARS,47765241100,27001,Transaction refused[02] [] [111800] [111800],New,13383677400,2018-05-10 01:04:20,01,Aut

In [15]:
import time
import datetime
from datetime import timedelta
#Convert from str to datetime
def to_date(datestr):
    struct = time.strptime(datestr, "%Y-%m-%d %H:%M:%S")
    date = datetime.date(struct.tm_year,struct.tm_mon,struct.tm_mday)
    return date

def fail_date_newer(row):
    try:
        fail_date = to_date(row['failed_attempt_date'])
        txn_date = to_date(row['transaction_date_in_string'])
    except Exception as ex:
        print("Found exception converting date for row: {}. Ex:{}".format(row, ex))
        return True
    return fail_date > txn_date

pdf_all['fail_date_newer'] = pdf_all.apply(fail_date_newer, axis=1)

In [1]:
%set_env SPARK_HOME=/opt/spark
import os
import findspark
os.environ['PYSPARK_SUBMIT_ARGS'] = """\
--master spark://10.62.42.243:7077
--jars "/drhome/hsanjaya/ojdbc6.jar" 
--packages datastax:spark-cassandra-connector:2.3.1-s_2.11,anguenot:pyspark-cassandra:0.9.0
--driver-memory 25g
--executor-memory 15g
--conf spark.local.dir=/var/spark
--conf spark.network.timeout=420000
--conf spark.shuffle.registration.timeout=42000
--conf spark.driver.cores=4
--conf spark.executor.cores=8
--conf spark.driver.maxResultSize=38
pyspark-shell
"""

import pyspark
from pyspark import SparkContext, SparkConf
import pandas as pd
findspark.init()
sc = pyspark.SparkContext(appName="EQUPRD")

sc._conf.set("spark.local.dir", "/var/spark")
sc._conf.set("spark.sql.execution.arrow.enabled", "true")

import pyspark
from pyspark.sql import SparkSession, SQLContext

spark = SparkSession.builder.\
    master("spark://10.62.42.243:7077").\
    appName("EQUPRD").\
    config(conf=sc._conf).\
    getOrCreate()
# spark

sqlContext = SQLContext(sc)
sqlContext

env: SPARK_HOME=/opt/spark


In [2]:
def _map_to_pandas(rdds):
    """ Needs to be here due to pickling issues """
    return [pd.DataFrame(list(rdds))]

def toPandas(df, n_partitions=None):
    """
    Returns the contents of `df` as a local `pandas.DataFrame` in a speedy fashion. The DataFrame is
    repartitioned if `n_partitions` is passed.
    :param df:              pyspark.sql.DataFrame
    :param n_partitions:    int or None
    :return:                pandas.DataFrame
    """
    if n_partitions is not None: df = df.repartition(n_partitions)
    df_pand = df.rdd.mapPartitions(_map_to_pandas).collect()
    df_pand = pd.concat(df_pand)
    df_pand.columns = df.columns
    return df_pand

expiration_date = '01/04/2019'
# query = "(select subscription_id as subscription_id, prod_dur as prod_dur, expiration_date as expiration_date, row_last_etl_date as row_last_etl_date from EREPORT.SUB_AUTH_ADDITIONAL_INFO where expiration_date <=  TO_DATE('{}','dd/mm/yyyy') )".format(expiration_date)
query = "(select subscription_id as subscription_id, prod_dur as prod_dur, expiration_date as expiration_date, row_last_etl_date as row_last_etl_date from EREPORT.SUB_AUTH_ADDITIONAL_INFO where expiration_date >  TO_DATE('{}','dd/mm/yyyy') )".format(expiration_date)

# subDF = spark.read \
#     .format("jdbc") \
#     .option("url", "jdbc:oracle:thin:@(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=dc3db74.dc3.digitalriver.com)(PORT=1580))(CONNECT_DATA=(SERVER=dedicated)(SID=equprd05)))") \
#     .option("dbtable", "SUB_SEG_EXPIRE_DNORM") \
#     .option("user", "hsanjaya") \
#     .option("password", "Prd==2019") \
#     .option("driver", "oracle.jdbc.driver.OracleDriver") \
#     .load()

url = "jdbc:oracle:thin:@(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=dc3db74.dc3.digitalriver.com)(PORT=1580))(CONNECT_DATA=(SERVER=dedicated)(SID=equprd05)))"

db_properties = {"user": "hsanjaya", "password": "Prd==2019", "driver": "oracle.jdbc.driver.OracleDriver"}
    
subDF = spark.read \
    .jdbc(url=url,table=query,properties=db_properties) 
    
subs_pdf =  toPandas(subDF, 64)   
# subs_pdf =  subDF.toPandas()

subs_pdf.shape

(18454818, 4)

In [3]:
WORK_DIR = '/var/spark/ml_files/'
subs_pdf_all = pd.read_csv(WORK_DIR + 'subs_prod_dur.csv')

In [4]:
subs_pdf_all = pd.concat([subs_pdf_all, subs_pdf], ignore_index=True)
subs_pdf_all.shape

/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  """Entry point for launching an IPython kernel.


(191663176, 6)

In [7]:
# subs_pdf_all.to_csv(WORK_DIR + 'subs_prod_dur.csv')

In [23]:
# WORK_DIR = '/var/spark/ml_files/'
# subs_pdf.to_csv(WORK_DIR + 'subs_prod_dur.csv')

In [5]:
subs_pdf_all = subs_pdf_all.drop_duplicates(subset='SUBSCRIPTION_ID', keep='last')
subs_pdf_all.shape

(32661612, 6)

In [6]:
WORK_DIR = '/var/spark/ml_files/'
subs_pdf_all.to_csv(WORK_DIR + 'subs_prod_dur_unique.csv')

In [19]:
# subDF.printSchema()
subs_pdf.shape

(9996037, 12)